In [1]:
import numpy as np
import sys
sys.path.append( '../vqe-in-dft' )
from tqdm import tqdm

import vqe_in_dft 

import scipy as sp
from pyscf import gto, dft, lib, mp, cc, scf, tools, ci, fci, lo, ao2mo

In [2]:
# geometry = [
# ['H', (0.7493682,0.0000000,0.4424329)],
# ['O', (0.0000000,0.0000000,-0.1653507)],
    
# ['H', (-0.7493682,0.0000000,0.4424329)]
# ]

geometry = [
['C', (0.0000, 0.0000, 0.0000)],
['H', (0.5288, 0.1610, 0.9359)],
['H', (0.2051, 0.8240, -0.6786)],
['H', (0.3345, -0.9314, -0.4496)],
['H', (-1.0685, -0.0537, 0.1921)]
]

# geometry = [
# ['H', (1.2473876659, -0.8998737590, 0.6150681570)],
# ['O', (1.2322305822, -0.2731895077, -0.1276123902)],

# ['C', (0.0849758188, 0.5590385475, 0.0510545434)],
# ['H', (0.1506137362, 1.1200249874, 0.9943015309)],
# ['H', (0.1316093068, 1.2841805400, -0.7645223601)],

# ['C', (-1.2129704155, -0.2295285634, -0.0097156258)],
# ['H', (-2.0801425360, 0.4329727646,0.0722817289)],
# ['H', (-1.2655910941, -0.9539857247, 0.8097953440)],
# ['H', (-1.2737541560, -0.7748626513, -0.9540587845)],
# ]

N_active_atoms = 2


low_level_scf_method='RKS'
high_level_scf_method='CCSD'
E_convergence_tol = 1e-6
basis = 'STO-3G' #'augccpvdz' # '6-31g' #'STO-3G'
unit= 'angstrom'
pyscf_print_level=1
memory=8000
charge=0
spin=0
run_fci= True#True
low_level_xc_functional = 'lda, vwn' # 'b3lyp'
high_level_xc_functional = 'b3lyp'

phys_notation = True

# Build the GLOBAL (full) molecule


and run supersystem calculation (cheap method)

https://pyscf.org/_modules/pyscf/gto/basis.html

In [544]:
full_system_mol = gto.Mole(atom= geometry,
                      basis=basis,
                       charge=charge,
                       unit=unit,
                       spin=spin,
                      )
full_system_mol.build()
full_system_mol.atom

[['C', (0.0, 0.0, 0.0)],
 ['H', (0.5288, 0.161, 0.9359)],
 ['H', (0.2051, 0.824, -0.6786)],
 ['H', (0.3345, -0.9314, -0.4496)],
 ['H', (-1.0685, -0.0537, 0.1921)]]

In [546]:
## FCI comparison
HF_scf = scf.RHF(full_system_mol)
HF_scf.verbose=1
HF_scf.max_memory= memory
HF_scf.conv_tol = 1e-6
HF_scf.kernel()

my_fci = fci.FCI(HF_scf).run()
print('E(UHF-FCI) = %.12f' % my_fci.e_tot)

# myci = ci.CISD(HF_scf).run() # this is UCISD
# print('UCISD total energy = ', myci.e_tot)

E(UHF-FCI) = -39.805676967111


In [545]:
# Draw molecule

xyz_string = vqe_in_dft.Get_xyz_string(full_system_mol)
vqe_in_dft.Draw_molecule(xyz_string, width=400, height=400, jupyter_notebook=True)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [547]:
# Run global calculation

full_system_scf = scf.RKS(full_system_mol)
full_system_scf.verbose= pyscf_print_level
full_system_scf.max_memory= memory
full_system_scf.conv_tol = 1e-6
full_system_scf.xc = low_level_xc_functional
full_system_scf.kernel()

-39.61684241519855

# Localise orbitals

### Background

The overlap matrix is:

$$S_{\mu \nu} =  \int d\vec{r}_{1} \phi_{\mu}(1)^{*}\phi_{\nu}(1)$$

- $\phi_{\mu}$ are basis functions (defined in basis set)


The unknown molecular orbitals $\psi_{i}$ are expanded as a linear expansion of the $K$ known basis functions:

$$ \psi_{i} =  \sum_{\mu=1}^{K} C_{\mu i} \psi_{\mu}$$


$C$ is a $K \times K$ matrix of expansion coefficients $C_{\mu i}$. The columns of $C$ describe the molecular orbitals!

We can find the total number of electrons $N$ in the system by:

$$ N =  2 \sum_{a}^{N/2}\int d\vec{r}  \bigg( \psi_{a}(\vec{r})^{*} \psi_{i}(\vec{r}) \bigg) =  2 \sum_{a}^{N/2} 1$$

- integral gives probablity of finding electron $a$ over all space (must be 1)
- summing over all electrons will give the total number of electrons

The charge density has the following definition:

$$\rho(\vec{r}) = 2 \sum_{a}^{N/2} \bigg( \psi_{a}(\vec{r})^{*} \psi_{i}(\vec{r}) \bigg)$$

- re-write using definition of $\psi_{i}=  \sum_{\mu=1}^{K} C_{\mu i} \phi_{\mu}$

$$\rho(\vec{r}) = 2 \sum_{a}^{N/2} \Bigg( \bigg[ \sum_{\nu}^{K} C_{\nu a}^{*} \phi_{\nu}(\vec{r})^{*} \bigg] \bigg[ \sum_{\mu}^{K} C_{\mu a}\phi_{\mu}(\vec{r}) \bigg] \Bigg)$$

- move things around

$$\rho(\vec{r}) = \sum_{\nu}^{K} \sum_{\mu}^{K} \Big( 2 \sum_{a}^{N/2} C_{\mu a} C_{\nu a}^{*} \Big) \phi_{\mu}(\vec{r}) \phi_{\nu}(\vec{r})^{*} $$

- which is 

$$\rho(\vec{r}) = \sum_{\mu, \nu}^{K} P_{\mu \nu} \phi_{\mu}(\vec{r}) \phi_{\nu}(\vec{r})^{*} $$


- $P_{\mu \nu}$ is known as the density matrix and is:

$$P_{\mu \nu} = 2 \sum_{a}^{N/2} C_{\mu a} C_{\nu a}^{*}$$

Therefore we can also find the total number of electrons in the system by:

$$ N =  2 \sum_{a}^{N/2}\int d\vec{r}  \bigg( \psi_{a}(\vec{r})^{*} \psi_{i}(\vec{r}) \bigg) =  \sum_{\nu}^{K} \sum_{\mu}^{K} \Big( 2 \sum_{a}^{N/2} C_{\mu a} C_{\nu a}^{*} \Big) \int d\vec{r} \phi_{\mu}(\vec{r})  \phi_{\nu}(\vec{r})^{*}$$

- This is simply:

$$N =  \sum_{\nu}^{K} \sum_{\mu}^{K} P_{\mu \nu} S_{\nu \mu}= \sum_{\mu}^{K} PS_{\mu \mu} = \mathcal{Tr}(PS)$$

- One can interpret $ PS_{\mu \mu}$ in the above equation as the number of electrons associated with $ \phi_{\mu}$
- This is a **Mulliken population analysis**

# Orbital Localization!

A molecular orbital is usually delocalized, i.e. it has non-negligible amplitude over the whole system rather than only around some atom(s) or bond(s). However, one can choose a unitary rotation 

- When we perform a SCF calculation, one gets an optimized C matrix
    - $C$ is a $K \times K$ matrix of expansion coefficients $C_{\mu i}$
    - The columns of $C$ describe the molecular orbitals!
    - MO i: $ \psi_{i} =  \sum_{\mu=1}^{K} C_{\mu i} \psi_{\mu}$
    
    
- These molecular orbitals are usually **delocalized**
    - non-negligible amplitude over the whole system, rather than only around some atom(s) or bond(s)

- But we know in QM that a given basis choice is NOT unique


- We can therefore perform a unitary rotation on molecular orbitals

$$ \psi_{i} U_{rot} =  \Big( \sum_{\mu=1}^{K} C_{\mu i} \psi_{\mu} \Big) U_{rot} = \psi_{i}^{new}$$
    
    
The idea is to use a rotation such that the resulting orbitals $\psi_{i}^{new}$ are as spatially localized as possible. 


The Pipek-Mezey (PM) [localization](https://notendur.hi.is/hj/papers/paperPipekmezey8.pdf) **maximizes the population charges on the atoms**:

$$ f (U_{rot}) = \sum_{A}^{N_{atoms}} \Bigg( Z_{A} -  \sum_{\mu \text{ on atom } A} PS_{\mu \mu} \Bigg)$$

# Choose active and environment systems

## METHOD 1 

- Given optimized $C$ coefficient matrix
    - which has been rotated to localize orbitals
    - (used to build localized density matrix)


- **Look through basis functions $\phi_{\mu}$ of the ACTIVE atoms**

    
- check the mulliken charge // mulliken population of the orbital
    - if above a certain threshold associate it to active system
    - otherwise put in the environment
 


To choose the active and enviroment subsystems we do the following:

1. Given a localized molecular orbs (localized C matrix), we take the absolute mag squared of the coefficients of the active part for a given localized orb and divide by the absolute mag squared of all the coefficents of a that orb... THis will give a value of how much the active system contributes to that orb.

2. Mathematically, for orbital $j$ 
    - remember MO orbs given by columns of C matrix
    - In equation below C matrix is the LOCALIZED form!


$$ \text{threshold} =  \frac{\sum_{\mu\in \text{active AO}}^{K} |C_{\mu j}|^{2}}{\sum_{\mu =1}^{K} |C_{\mu j}|^{2}}$$

## METHOD 2 - SPADE

WRITE NOTES TODO

In [548]:
def Localize_orbitals(localization_method, PySCF_scf_obj, N_active_atoms, THRESHOLD=None, sanity_check=True):

    if PySCF_scf_obj.mo_coeff is None:
        raise ValueError('need to perform SCF calculation before localization')


    S_ovlp = PySCF_scf_obj.get_ovlp()
    AO_slice_matrix = PySCF_scf_obj.mol.aoslice_by_atom()

    occupied_orbs = PySCF_scf_obj.mo_coeff[:,PySCF_scf_obj.mo_occ>0]
    # run localization scheme
    if localization_method.lower() == 'spade':
        

        # Get active AO indices
        N_active_AO = AO_slice_matrix[N_active_atoms-1][3]  # find max AO index for active atoms (neg 1 as python indexs from 0)

        
        S_half = sp.linalg.fractional_matrix_power(S_ovlp , 0.5)
        orthogonal_orbitals = (S_half@occupied_orbs)[:N_active_AO, :] # Get rows (the active AO) of orthogonal orbs 

        # Comute singular vals
        u, singular_values, rotation_matrix = np.linalg.svd(orthogonal_orbitals, full_matrices=True)

        # find where largest step change 
        delta_s = singular_values[:-1] - singular_values[1:] # σ_i - σ_(i+1)
        print('delta_singular_vals:')
        print(delta_s, '\n')

        n_act_mos = np.argmax(delta_s)+1 # add one due to python indexing
        n_env_mos = len(singular_values) - n_act_mos

        # define active and environment orbitals from localization
        act_orbitals = occupied_orbs @ rotation_matrix.T[:, :n_act_mos]
        env_orbitals = occupied_orbs @ rotation_matrix.T[:, n_act_mos:]

        C_matrix_all_localized_orbitals = occupied_orbs @ rotation_matrix.T

        active_MO_inds  = np.arange(n_act_mos)
        enviro_MO_inds = np.arange(n_act_mos, n_act_mos+n_env_mos)

    else:
        if not isinstance(THRESHOLD, float):
            raise ValueError ('if localization method is not SPADE then a threshold parameter is requried to choose active system')


        # run localization scheme
        if localization_method.lower() == 'pipekmezey':
            ### PipekMezey
            PM = lo.PipekMezey(PySCF_scf_obj.mol, occupied_orbs)
            PM.pop_method = 'mulliken' # 'meta-lowdin', 'iao', 'becke'
            C_loc_occ = PM.kernel() # includes virtual orbs too!

        elif localization_method.lower() == 'boys':
            ### Boys
            boys_SCF = lo.boys.Boys(PySCF_scf_obj.mol, occupied_orbs)
            C_loc_occ  = boys_SCF.kernel()

        elif localization_method.lower() == 'ibo':
            ### intrinsic bonding orbs
            #
            iaos = lo.iao.iao(PySCF_scf_obj.mol, occupied_orbs)
            # Orthogonalize IAO
            iaos = lo.vec_lowdin(iaos, S_ovlp)
            C_loc_occ = lo.ibo.ibo(PySCF_scf_obj.mol, occupied_orbs, locmethod='IBO', iaos=iaos)#.kernel()

            # iaos = lo.iao.iao(PySCF_scf_obj.mol, PySCF_scf_obj.mo_coeff)
            # # Orthogonalize IAO
            # iaos = lo.vec_lowdin(iaos, S_ovlp)
            # C_loc = lo.ibo.ibo(PySCF_scf_obj.mol, PySCF_scf_obj.mo_coeff, locmethod='IBO', iaos=iaos)#.kernel()
            # C_loc_occ = C_loc[:,PySCF_scf_obj.mo_occ>0]
        else:
            raise ValueError(f'unknown localization method {localization_method}')
        
                        
        # find indices of AO of active atoms
        ao_active_inds = np.arange(AO_slice_matrix[0,2], AO_slice_matrix[N_active_atoms-1,3])

        ### New method
        numerator_all = np.einsum('ij->j', (C_loc_occ[ao_active_inds, :])**2) # active AOs coeffs for a given MO j
        denominator_all = np.einsum('ij->j', C_loc_occ**2) # all AOs coeffs for a given MO j

        MO_active_percentage = numerator_all/denominator_all

        print('\n(active_AO^2)/(all_AO^2):', np.around(MO_active_percentage,4))
        print(f'threshold for active part: {THRESHOLD} \n')

        active_MO_inds = np.where(MO_active_percentage>THRESHOLD)[0]
        enviro_MO_inds = np.array([i for i in range(C_loc_occ.shape[1]) if i not in active_MO_inds]) # get all non active MOs


        # define active MO orbs and environment
        act_orbitals = C_loc_occ[:, active_MO_inds] # take MO (columns of C_matrix) that have high dependence from active AOs
        env_orbitals = C_loc_occ[:, enviro_MO_inds]
        
        C_matrix_all_localized_orbitals = C_loc_occ

        n_act_mos = len(active_MO_inds)
        n_env_mos = len(enviro_MO_inds)

    print(f'number of active MOs: {n_act_mos}')
    print(f'number of enviro MOs: {n_env_mos} \n')

    return act_orbitals, env_orbitals, C_matrix_all_localized_orbitals, active_MO_inds, enviro_MO_inds # C_active, C_enviro, C_all_localized, active_MO_inds, enviro_MO_inds

In [549]:
localization_method= 'PipekMezey' # 'ibo', 'Boys', 'PipekMezey' 'SPADE'
THRESHOLD = 0.95

(C_active, 
 C_envrio, 
 C_all_localized, 
 active_MO_inds,
 enviro_MO_inds) = Localize_orbitals(localization_method, 
                                     full_system_scf, 
                                     N_active_atoms, 
                                     THRESHOLD=THRESHOLD, 
                                     sanity_check=True)



(active_AO^2)/(all_AO^2): [0.9988 0.5689 0.9855 0.5689 0.5689]
threshold for active part: 0.95 

number of active MOs: 2
number of enviro MOs: 3 



In [550]:
dm_loc = 2* C_all_localized@ C_all_localized.conj().T
dm_std = 2* full_system_scf.mo_coeff[:,full_system_scf.mo_occ>0]@ full_system_scf.mo_coeff[:,full_system_scf.mo_occ>0].conj().T

print('want these C_loc and C_opt_std to be different:', not np.allclose(C_all_localized,
                                                                         full_system_scf.mo_coeff[:,full_system_scf.mo_occ>0]))
print('want dm_std and dm_loc to be the SAME:',np.allclose(dm_std, dm_loc))

want these C_loc and C_opt_std to be different: True
want dm_std and dm_loc to be the SAME: True


Virtual localized orbs!:
https://pyscf.org/_modules/pyscf/lo/vvo.html

In [551]:
from pyscf.lo import vvo

orbocc = full_system_scf.mo_coeff[:,full_system_scf.mo_occ>0]
orbvirt = full_system_scf.mo_coeff[:,full_system_scf.mo_occ<2]
C_virtual_loc = vvo.vvo(full_system_scf.mol,
                    orbocc,
                    orbvirt,
                    iaos=None, 
                    s=None,
                    verbose=None)

C_virtual_loc.shape

# could truncate here using active MO again!!!

# PUT localized virtual orbs into full C matrix!
C_all_localized_and_virt = np.hstack((C_all_localized,
                                     C_virtual_loc))

In [552]:
# PUT standard virtual orbs into full C matrix (no change from standard HF virutal orbs)!
C_all_localized_and_virt = np.hstack((C_all_localized,
                                     full_system_scf.mo_coeff[:,full_system_scf.mo_occ<2]))

In [553]:
S_half = sp.linalg.fractional_matrix_power(full_system_scf.get_ovlp() , 0.5)
test =  S_half@ C_virtual_loc
test[:,0].dot(test[:,2])


-1.27675647831893e-15

# IMPORTANT orbital localization point!

When we performed orbital localization a unitary rotation was used... this left the generated density matrix (C * C) unchanged...

HOWEVER we need to also **rotate the Fock matrix** otherwise the orbital energies no longer make any sense (even though the final energy will be the same)!


In [554]:
# here we see the localized and standard density matrices are the same!
print('want dm_std and dm_loc to be the SAME:',np.allclose(dm_std, dm_loc), '\n')


print('standard orb energies : \n')
print(np.diag(full_system_scf.mo_coeff.conj().T @ full_system_scf.get_fock() @ full_system_scf.mo_coeff))
print(full_system_scf.mo_energy)

print('\nLOCALIZED orb energies : \n')
print(np.diag(C_all_localized_and_virt.conj().T @ full_system_scf.get_fock() @ C_all_localized_and_virt))
print(full_system_scf.mo_energy)

# NOTE HOW THERE IS A DIFFERENCE WHEN LOCALIZED... To fix this we need to rotate the Fock matrix!

want dm_std and dm_loc to be the SAME: True 

standard orb energies : 

[-9.56763009 -0.58148699 -0.30276035 -0.3027371  -0.30273108  0.4253797
  0.42539908  0.42544391  0.47125932]
[-9.56762769 -0.58148626 -0.30275977 -0.30273646 -0.30273044  0.42538041
  0.4253998   0.42544456  0.4712599 ]

LOCALIZED orb energies : 

[-9.5475474  -0.37745285 -0.37746327 -0.37743824 -0.37744386  0.4253797
  0.42539908  0.42544391  0.47125932]
[-9.56762769 -0.58148626 -0.30275977 -0.30273646 -0.30273044  0.42538041
  0.4253998   0.42544456  0.4712599 ]


How to get the operator that changes the basis?

1. Put the standard (unlocalized) and localized C matrix into orthogonal basis
    - $C_{\text{std}}^{\text{ORTHO}} = S^{0.5}C_{\text{std}}$
    - $C_{\text{loc}}^{\text{ORTHO}} = S^{0.5}C_{\text{loc}}$

2. Define change of basis
    - $U = \sum_{i \in occupied} | \psi_{i}^{\text{ORTHO, STD}} \rangle \langle \psi_{i}^{\text{ortho, loc}} | + \sum_{i \in virtual} | \psi_{i}^{\text{ORTHO, STD}} \rangle \langle \psi_{i}^{\text{ortho, STD}}| $ 
    - note this projects occupied standard ortho orbs onto localized orbs
    - keeps virtual orbs the same
        - if we localize the virtual orbs then we would need to use the same procedure as the occupied orbs (easy to include)

In [555]:
# note can make this much better with einsum!

S_mat = full_system_scf.get_ovlp()
S_half = sp.linalg.fractional_matrix_power(full_system_scf.get_ovlp() , 0.5)

ortho_std = S_half@ full_system_scf.mo_coeff
ortho_loc = S_half@ C_all_localized_and_virt

# ortho_loc[:,0].dot(ortho_loc[:,2])

U = np.zeros((ortho_std.shape[0], ortho_std.shape[0]))
for MO_ind in range(ortho_std.shape[1]):
    outer = np.outer(ortho_std[:, MO_ind], ortho_loc[:, MO_ind])
    U+=outer

print(f'is U*C_ortho_loc =  C_ortho_STD: {(np.allclose(U @ ortho_loc, ortho_std))}')
print(f'is U Udag = Identity: {np.allclose(U.conj().T@U, np.eye(U.shape[0]))}')
                                          
# ind = 0
# Fock = full_system_scf.get_fock(dm=dm_std)
# print(ortho_loc[:,ind].conj().T @ (U.conj().T @ Fock @ U) @ ortho_loc[:,ind])
# print(ortho_std[:,ind].conj().T  @ Fock @ ortho_std[:,ind])

is U*C_ortho_loc =  C_ortho_STD: True
is U Udag = Identity: True


Note that $U$ is currently defined using ORTHOGONAL BASIS... we want it in our usual MO basis

Remembering that

$$C_{\text{std}}^{\text{ORTHO}} = S^{0.5}C_{\text{std}}$$

Clearly to get back to standard basis we do:

$$C_{\text{std}} = S^{-0.5} C_{\text{std}}^{\text{ORTHO}}$$


NOW as 


$$U C_{\text{loc}}^{\text{ORTHO}} =C_{\text{std}}^{\text{ORTHO}}$$

we can re-write this using our definitions:
- $C_{\text{std}}^{\text{ORTHO}} = S^{0.5}C_{\text{std}}$
- $C_{\text{loc}}^{\text{ORTHO}} = S^{0.5}C_{\text{loc}}$

$$U (S^{0.5}C_{\text{std}}) = (S^{0.5}C_{\text{loc}})$$

Now we want to get rid of $S^{0.5}$ on RHS so we times on left by $S^{-0.5}$

$$ S^{-0.5} U S^{0.5}C_{\text{std}} = C_{\text{loc}}$$

Therefore the change of basis operator in our non-ortho basis is simply:

$$ S^{-0.5} U S^{0.5}C_{\text{std}} = V$$

Where: $ V C_{\text{std}} = C_{\text{loc}}$


In [15]:
S_neg_half = sp.linalg.fractional_matrix_power(full_system_scf.get_ovlp() , -0.5)

V = S_neg_half @ U @ S_half


print('V @ C_loc = C_std:', np.allclose(V@C_all_localized_and_virt,
                                            full_system_scf.mo_coeff))

V @ C_loc = C_std: True


NOW we can check orbital energies of localized system when we use V to change basis:

In [556]:
print('standard orb energies : \n')
print(np.diag(full_system_scf.mo_coeff.conj().T @ full_system_scf.get_fock() @ full_system_scf.mo_coeff))
print(full_system_scf.mo_energy)

print('\nLOCALIZED orb energies WITHOUT Fock change of basis : \n')
print(np.diag(C_all_localized.conj().T @ full_system_scf.get_fock() @ C_all_localized))
print(full_system_scf.mo_energy)


print('\nLOCALIZED orb energies WITH Fock change of basis : \n')
print(np.diag(C_all_localized_and_virt.conj().T @ (V.conj().T @ full_system_scf.get_fock() @ V) @ C_all_localized_and_virt))
print(full_system_scf.mo_energy)

standard orb energies : 

[-9.56763009 -0.58148699 -0.30276035 -0.3027371  -0.30273108  0.4253797
  0.42539908  0.42544391  0.47125932]
[-9.56762769 -0.58148626 -0.30275977 -0.30273646 -0.30273044  0.42538041
  0.4253998   0.42544456  0.4712599 ]

LOCALIZED orb energies WITHOUT Fock change of basis : 

[-9.5475474  -0.37745285 -0.37746327 -0.37743824 -0.37744386]
[-9.56762769 -0.58148626 -0.30275977 -0.30273646 -0.30273044  0.42538041
  0.4253998   0.42544456  0.4712599 ]

LOCALIZED orb energies WITH Fock change of basis : 

[-9.56763009 -0.58148699 -0.30276035 -0.3027371  -0.30273108  0.4253797
  0.42539908  0.42544391  0.47125932]
[-9.56762769 -0.58148626 -0.30275977 -0.30273646 -0.30273044  0.42538041
  0.4253998   0.42544456  0.4712599 ]


In [944]:
# modify H_core and Veff to allow this change of basis!!!

def Get_new_Hcore(H_core, Unitary_rot):
    H_core_rot = Unitary_rot.conj().T @ H_core @Unitary_rot 
    return H_core_rot

from pyscf.dft.rks import get_veff as RKS_get_veff
## RKS_get_veff gives Coulomb + XC functional
## in matrix form: Veff = J + Vxc.
    
from pyscf import lib
from pyscf.dft import numint

def Get_new_Veff(pyscf_obj, Unitary, dm=None, check_result=False):
    
    if dm is None:
        if pyscf_obj.mo_coeff is not None:
            density_mat = pyscf_obj.make_rdm1(pyscf_obj.mo_coeff, pyscf_obj.mo_occ)
        else:
            density_mat = pyscf_obj.init_guess_by_1e()
    else:
        density_mat = dm
    
    
#     Evaluate RKS/UKS XC functional and potential matrix on given meshgrids
#     for a set of density matrices.
    nelec, exc, vxc = numint.nr_vxc(pyscf_obj.mol,
                                            pyscf_obj.grids,
                                            pyscf_obj.xc,
                                            density_mat)
    vxc =  Unitary.conj().T @ vxc @ Unitary
    
    Veff = RKS_get_veff(pyscf_obj, dm=density_mat)
    if Veff.vk is not None:
        K = Unitary.conj().T @ Veff.vk @ Unitary
        J = Unitary.conj().T @ Veff.vj @ Unitary
        vxc += J - K * .5
    else:
        J = Unitary.conj().T @ Veff.vj @ Unitary
        K = None
        vxc += J 
    
    if check_result is True:
        M1 = Unitary.conj().T @ Veff.__array__() @ Unitary
        if not np.allclose(vxc, M1):
            raise ValueError('Veff in new basis NOT correct')
    
    ecoul = np.einsum('ij,ji', density_mat, J).real * .5 # note J matrix is in new basis!
    ## this ecoul term changes if the full density matrix is NOT 
    # (aka for dm_active and dm_enviroment we get different V_eff under different bases!)
    
    
    output = lib.tag_array(vxc, ecoul=ecoul, exc=Veff.exc, vj=J, vk=K)
    return output


In [18]:
# Get_new_Veff(full_system_scf,
#                U_new,
#             check_result=True)

In [19]:
# from pyscf.dft import numint
# # manual version of
# # https://pyscf.org/_modules/pyscf/dft/dks.html#DKS.x2c1e
# nelec, exc, vxc = numint.nr_vxc(test_full_system_scf.mol,
#                                     test_full_system_scf.grids,
#                                     test_full_system_scf.xc,
#                                     dm_loc)

# J= test_full_system_scf.get_j(test_full_system_scf.mol, dm_loc)
# vxc += J
# vxc

# Veff = RKS_get_veff(test_full_system_scf, dm=dm_loc)
# np.allclose(Veff.__array__(), vxc)

In [20]:
# Veff = RKS_get_veff(test_full_system_scf, dm=dm_loc)
# Veff_rot = U_new.conj().T @ Veff.__array__() @ U_new

# test = Get_new_Veff(test_full_system_scf, U_new)
# np.allclose(test, Veff_rot)

In [950]:
# Check pyscf obj with localized orbs and unitary change of basis gives same results as before!

test_full_system_scf = scf.RKS(full_system_mol)
test_full_system_scf.verbose= pyscf_print_level
test_full_system_scf.max_memory= memory
test_full_system_scf.conv_tol = 1e-6
test_full_system_scf.xc = low_level_xc_functional


Hcore_std = test_full_system_scf.get_hcore()
test_full_system_scf.get_hcore = lambda *args: Get_new_Hcore(Hcore_std, 
                                                             Unitary_rot=V)


test_full_system_scf.get_veff = lambda mol=None, dm=None, dm_last=0, vhf_last=0, hermi=1: Get_new_Veff(
                                                                                    test_full_system_scf,
                                                                                       V,
                                                                                        dm=dm)


test_full_system_scf.mo_coeff = C_all_localized_and_virt
test_full_system_scf.mo_occ = full_system_scf.mo_occ

test_full_system_scf.energy_tot(dm=dm_loc)

-39.61684241524395

In [559]:
print(np.allclose(C_std.conj().T @ full_system_scf.get_hcore() @ C_std,
           C_all_localized_and_virt.conj().T @ test_full_system_scf.get_hcore() @ C_all_localized_and_virt))

print(np.allclose(C_std.conj().T @ full_system_scf.get_veff(dm=dm_std) @ C_std,
           C_all_localized_and_virt.conj().T @ test_full_system_scf.get_veff(dm=dm_loc) @ C_all_localized_and_virt))

True
True


In [560]:
### RKS energy (not the same as HFock!)
vhf_std = full_system_scf.get_veff(dm=dm_std)
e1_std = np.einsum('ij,ji->', full_system_scf.get_hcore(), dm_std)
e2_std = vhf_std.ecoul + vhf_std.exc
print(e1_std + e2_std)

print()
vhf_loc = test_full_system_scf.get_veff(dm=dm_std)
e1_loc = np.einsum('ij,ji->', test_full_system_scf.get_hcore(), dm_loc)
e2_loc = vhf_loc.ecoul + vhf_loc.exc
print(e1_loc+e2_loc)

-53.08968539169484

-53.08968539174026


In [561]:
print('check electronic energies match:', np.allclose(full_system_scf.energy_elec(dm=dm_std)[0],
                                test_full_system_scf.energy_elec(dm=dm_loc)[0]))

# test_full_system_scf.kernel()  # run to populate mo_energy....
# print('check MO energies match:', np.allclose(full_system_scf.mo_energy,
#                                 test_full_system_scf.mo_energy))

check electronic energies match: True


In [562]:
F_new = test_full_system_scf.get_hcore() + test_full_system_scf.get_veff(dm_loc) # Note this is now in new basis!
print(np.around(C_all_localized_and_virt.conj().T @ F_new @ C_all_localized_and_virt, 2))


F_std = full_system_scf.get_hcore() + full_system_scf.get_veff(dm=dm_loc) # Note this is standard HF!
C_std = full_system_scf.mo_coeff
print(np.around(C_std.conj().T @ F_std @ C_std, 2))

[[-9.57  0.    0.   -0.    0.    0.   -0.   -0.    0.  ]
 [ 0.   -0.58  0.   -0.    0.   -0.    0.    0.   -0.  ]
 [ 0.    0.   -0.3  -0.    0.    0.    0.    0.   -0.  ]
 [-0.   -0.   -0.   -0.3   0.    0.   -0.    0.    0.  ]
 [ 0.    0.    0.    0.   -0.3  -0.   -0.    0.   -0.  ]
 [ 0.   -0.    0.    0.   -0.    0.43  0.   -0.    0.  ]
 [-0.    0.    0.   -0.   -0.    0.    0.43  0.   -0.  ]
 [-0.    0.    0.    0.    0.   -0.    0.    0.43 -0.  ]
 [ 0.   -0.   -0.    0.   -0.    0.   -0.   -0.    0.47]]
[[-9.57  0.    0.   -0.    0.    0.   -0.   -0.    0.  ]
 [ 0.   -0.58  0.   -0.    0.   -0.    0.    0.   -0.  ]
 [ 0.    0.   -0.3  -0.    0.    0.    0.    0.   -0.  ]
 [-0.   -0.   -0.   -0.3   0.    0.   -0.    0.    0.  ]
 [ 0.    0.    0.    0.   -0.3  -0.   -0.    0.   -0.  ]
 [ 0.   -0.    0.    0.   -0.    0.43  0.   -0.    0.  ]
 [-0.    0.    0.   -0.   -0.    0.    0.43  0.   -0.  ]
 [-0.    0.    0.    0.    0.   -0.    0.    0.43 -0.  ]
 [ 0.   -0.   -0.    0.   -0. 

Basically we now have orbital energies matching as we want

(This will become important when we move to post Hartree-Fock methods!)

In [ ]:
# # Draw active orbitals

# list_active_orbitals = vqe_in_dft.Draw_cube_orbital(full_system_scf,
#                              xyz_string,
#                              C_all_localized, 
#                              active_MO_inds,
#                              width=400, 
#                              height=400, 
#                              jupyter_notebook=True)

# list_enviro_orbitals = vqe_in_dft.Draw_cube_orbital(full_system_scf,
#                              xyz_string,
#                              C_all_localized, 
#                              enviro_MO_inds,
#                              width=400, 
#                              height=400, 
#                              jupyter_notebook=True)

In [ ]:
# list_active_orbitals[0]

In [ ]:
# list_enviro_orbitals[0]

In [563]:
def Get_active_and_envrio_dm(PySCF_scf_obj, C_active, C_envrio, C_all_localized, sanity_check=True):

    # get density matrices
    dm_active =  2 * C_active @ C_active.T
    dm_enviro =  2 * C_envrio @ C_envrio.T

    if sanity_check:

        S_ovlp = PySCF_scf_obj.get_ovlp()

        ## check number of electrons is still the same after orbitals have been localized (change of basis)
        N_active_electrons = np.trace(dm_active@S_ovlp)
        N_enviro_electrons = np.trace(dm_enviro@S_ovlp)
        N_all_electrons = PySCF_scf_obj.mol.nelectron

        bool_flag_electron_number = np.isclose(( N_active_electrons + N_enviro_electrons), N_all_electrons)
        if not bool_flag_electron_number:
            raise ValueError('number of electrons in localized orbitals is incorrect')
        print(f'N_active_elec + N_environment_elec = N_total_elec is: {bool_flag_electron_number}')

        # checking denisty matrix parition makes sense:
        dm_localised_full_system = 2* C_all_localized@ C_all_localized.conj().T
        bool_density_flag = np.allclose(dm_localised_full_system, dm_active + dm_enviro)
        if not bool_density_flag:
            raise ValueError('gamma_full != gamma_active + gamma_enviro')
        print(f'y_active + y_enviro = y_total is: {bool_density_flag}')

    return dm_active, dm_enviro

In [564]:
dm_active, dm_enviro = Get_active_and_envrio_dm(
                                            full_system_scf,
                                            C_active, 
                                            C_envrio, 
                                            C_all_localized, 
                                            sanity_check=True)

N_active_elec + N_environment_elec = N_total_elec is: True
y_active + y_enviro = y_total is: True


In [565]:
two_e_term =  test_full_system_scf.get_veff(dm=dm_loc)

np.allclose(V.conj().T @test_full_system_scf.get_j(dm = dm_loc) @ V,
            two_e_term.vj)

True

In [566]:
def Get_energy_and_matrices_from_dm(PySCF_scf_obj, dm_matrix, check_E_with_pyscf=True):
    """
    Get Energy from denisty matrix

    Note this uses the standard hcore (NO embedding potential here!)
    """

    # It seems that PySCF lumps J and K in the J array 
    two_e_term =  PySCF_scf_obj.get_veff(dm=dm_matrix)
    J_mat = two_e_term.vj
    K_mat = np.zeros_like(J_mat)
    
    e_xc = two_e_term.exc
    v_xc = two_e_term - J_mat 

#     H_core_standard = scf.hf.get_hcore(PySCF_scf_obj.mol) # No embedding potential
    Energy_elec = (np.einsum('ij,ji->', PySCF_scf_obj.get_hcore(), dm_matrix) + 
                   two_e_term.ecoul + two_e_term.exc)
    
    if check_E_with_pyscf:
        Energy_elec_pyscf = PySCF_scf_obj.energy_elec(dm=dm_matrix)[0]
        if not np.isclose(Energy_elec_pyscf, Energy_elec):
            raise ValueError('Energy calculation incorrect')

    return Energy_elec, J_mat, K_mat, e_xc, v_xc

In [567]:
# use active density
E_act, J_act, K_act, e_xc_act, v_xc_act = Get_energy_and_matrices_from_dm( full_system_scf, # <-- standard
                                                                             dm_active, # <- ACTIVE
                                                                             check_E_with_pyscf=True)
print(full_system_scf.energy_elec(dm=dm_active)[0])
E_act

-42.315152602312835


-42.315152602312835

In [568]:
# use active density
E_act, J_act, K_act, e_xc_act, v_xc_act = Get_energy_and_matrices_from_dm( test_full_system_scf, # <-- modified 
                                                                             dm_active, # <- ACTIVE
                                                                             check_E_with_pyscf=True)
print(test_full_system_scf.energy_elec(dm=dm_active)[0])
E_act

-42.34501663118313


-42.34501663118313

In [569]:
## Note different energies!
print(full_system_scf.energy_elec(dm=dm_active)[0])
print(test_full_system_scf.energy_elec(dm=dm_active)[0])

## This is expected as now using active density matrix (rather than full one)

print()
# If we use the full dm matrix then we get matching results (aka now have full density matrix)
E_loc = test_full_system_scf.energy_elec(dm=dm_active+dm_enviro)[0]
E_std = full_system_scf.energy_elec(dm=dm_active+dm_enviro)[0] 
print(E_loc)
print(E_std)

-42.315152602312835
-42.34501663118313

-53.08968539174027
-53.08968539169487


In [570]:
# Note we want to use modified object, where change of basis on Fock matrix has been performed!

In [571]:
# use enviro density
E_env, J_env, K_env, e_xc_env, v_xc_env = Get_energy_and_matrices_from_dm(
                                                                            test_full_system_scf, # <-- modified 
                                                                             dm_enviro, # <- ENVIRO
                                                                             check_E_with_pyscf=True)
print(test_full_system_scf.energy_elec(dm=dm_enviro)[0])
E_env

-23.781447923243903


-23.781447923243903

In [572]:
def Get_cross_terms(PySCF_scf_obj, dm_active, dm_enviro, J_env, J_act, e_xc_act, e_xc_env):
    """
    Get cross system terms
    
    """

    two_e_term_total =  PySCF_scf_obj.get_veff(dm=dm_active+dm_enviro)
    e_xc_total = two_e_term_total.exc

    j_cross = 0.5 * ( np.einsum('ij,ij', dm_active, J_env) + np.einsum('ij,ij', dm_enviro, J_act) )
    k_cross = 0.0

    xc_cross = e_xc_total - e_xc_act - e_xc_env
    two_e_cross = j_cross + k_cross + xc_cross
    
    return two_e_cross

In [573]:
# cross terms!
two_e_cross = Get_cross_terms(test_full_system_scf, # <-- modified 
                                         dm_active, 
                                         dm_enviro, 
                                         J_env, 
                                         J_act, 
                                         e_xc_act,
                                         e_xc_env)

In [574]:
print(f'E_active: {E_act}')
print(f'E_enviro: {E_env}')
print(f'E_cross: {two_e_cross}')

E_active: -42.34501663118313
E_enviro: -23.781447923243903
E_cross: 13.036779162686791


In [575]:
print(f'E_hf_standard : {full_system_scf.e_tot}')
print(f'E_act + E_env + non_add_two_e + E_nuc : {E_act+E_env+two_e_cross+ full_system_scf.energy_nuc()}')

# equation 1 of SPADE paper
print(f'energies match {np.isclose(full_system_scf.e_tot, (E_act+E_env+two_e_cross+ full_system_scf.energy_nuc()))}') # equation 3 of SPADE paper
full_system_scf.e_tot - (E_act+E_env+two_e_cross+ full_system_scf.energy_nuc())

E_hf_standard : -39.61684241519855
E_act + E_env + non_add_two_e + E_nuc : -39.616842415243944
energies match True


4.53965753877128e-11

In [576]:
print(np.trace(S_mat@dm_enviro))
print(np.trace(S_mat@dm_active))

6.000000000000006
4.000000000000003


# NEXT define the projector onto subsystem B (enviroment)

$$P^{env} = \sum_{i \in \text{env}} |\psi_{i} \rangle \langle \psi_{i}|$$

important points:
1. Currently C_localized matrix columns give MO orbs $|\psi_{i} \rangle$
2. However, these are **not orthogonal**! Therefore defining a projector here is HARD
3. To fix this:
    - change to orthogonal basis
    - now define projector in this new basis
    - undo basis change to give projector in non-ortho basis!

In [962]:
## manual projector

## 1. convert to orthogonal C_matrix
S_mat = full_system_scf.get_ovlp()
S_half = sp.linalg.fractional_matrix_power(full_system_scf.get_ovlp() , 0.5)
S_neg_half = sp.linalg.fractional_matrix_power(full_system_scf.get_ovlp() , -0.5)

Loc_Ortho = S_half@ C_all_localized_and_virt # orthogonal C matrix (localized)

## 2. Define projector that projects MO orbs of subsystem B onto themselves and system A onto zero state!
##### (do this in orthongoal C_matrix!)
### not we only take MO environment indices!
PROJ_ortho = np.zeros_like(S_mat)
for MO_ind in range(C_all_localized_and_virt.shape[1]):
    if MO_ind in enviro_MO_inds:
        outer = np.outer(Loc_Ortho[:, MO_ind], Loc_Ortho[:, MO_ind])
        PROJ_ortho+=outer
    else:
        continue

print(f'''Are subsystem B (env) projected onto themselves: {
        np.allclose(PROJ_ortho@Loc_Ortho[:, enviro_MO_inds], 
        Loc_Ortho[:, enviro_MO_inds])}''') # projected onto itself

print(f'''Is subsystem A traced out?: {
        np.allclose(PROJ_ortho@Loc_Ortho[:, active_MO_inds], 
        np.zeros_like(Loc_Ortho[:, active_MO_inds]))}''') # # projected onto zeros!
        
        
#### 3. NEXT USING PROJECTOR in ortho basis define projector via different methods
        
        
# projector_method = 'huzinaga'
projector_method = 'mu_shfit'

if projector_method == 'huzinaga':
    Fock = test_full_system_scf.get_hcore() + test_full_system_scf.get_veff(dm=dm_active + dm_enviro)
    Fock_ortho = S_neg_half@Fock @ S_neg_half
    
    # Huzinaga
    projector_ortho =  -0.5*(Fock_ortho@PROJ_ortho + PROJ_ortho@Fock_ortho) # 0.5 for restricted calc!
    
    projector = S_half @ projector_ortho  @ S_half
    
elif projector_method == 'mu_shfit':
    mu = 1e6
    projector_ortho = mu * PROJ_ortho
    
#     projector = S_neg_half @ projector_ortho  @ S_half

    # mu shift
    projector = S_half @ projector_ortho  @ S_half
else:
    raise ValueError(f'Unknown projection method {projector_method}')

##### 4. Define projector in standard (non-orthogonal basis)
# projector = S_half @ projector_ortho  @ S_half


### 5. now can build relevant ops



### TODO check this!!!
# when using full_system_scf and mu_shift get correct result
# but I think one should use test_full_system_scf (with modified Fock matrix) 

### NOTE: we do NOT use modified Fock matrix here!!!!!
# g_A_and_B = full_system_scf.get_veff(dm=dm_active+dm_enviro)
# g_A = full_system_scf.get_veff(dm=dm_active)

# modified Fock matrix gives correct result when change of basis used on embedded pyscf obj!!!
g_A_and_B = test_full_system_scf.get_veff(dm=dm_active+dm_enviro)
g_A = test_full_system_scf.get_veff(dm=dm_active)

v_emb = g_A_and_B - g_A + projector

Are subsystem B (env) projected onto themselves: True
Is subsystem A traced out?: True


In [963]:
Fock_loc_basis = test_full_system_scf.get_fock()

if projector_method == 'mu_shfit':
    new_orbital_energies = np.diag(C_all_localized_and_virt.conj().T @ (Fock_loc_basis+projector) @ C_all_localized_and_virt)

    ## expect subsystem A orbital energies to remain the SAME
    print(f'''active subsystem orbital energies the same: {np.allclose(new_orbital_energies[active_MO_inds],
                                                        full_system_scf.mo_energy[active_MO_inds])} \n''')

    # expect subsystem B orbital energies to be pushed up to high energy
    print(f'original MO energies env: {full_system_scf.mo_energy[enviro_MO_inds]}')
    print(f'new MO energies env: {new_orbital_energies[enviro_MO_inds]}')
    # print(f'''enviro subsystem orbital energies energy increased by mu: 
    #                                                     {np.allclose(new_orbital_energies[enviro_MO_inds],
    #                                                     mu*full_system_scf.mo_energy[enviro_MO_inds])}''')
    
    
elif projector_method == 'huzinaga':
    ## projector should give opposite sign val of subsytem B orbitals vs standard Fock eval
    proj_orb_energy = C_all_localized_and_virt[:, enviro_MO_inds].conj().T @projector@C_all_localized_and_virt[:, enviro_MO_inds]
    fock_orb_energy = C_all_localized_and_virt[:, enviro_MO_inds].conj().T @Fock_loc_basis@C_all_localized_and_virt[:, enviro_MO_inds]
    
    print(f''' <env| P |env> = - <env| F |env> : {np.allclose(-1*proj_orb_energy,
                                                              fock_orb_energy)} \n''')
    
    new_orbital_energies = np.diag(C_all_localized_and_virt.conj().T @ (Fock_loc_basis+projector) @ C_all_localized_and_virt)
    print(f'''active subsystem orbital energies the same: {np.allclose(new_orbital_energies[active_MO_inds],
                                                    full_system_scf.mo_energy[active_MO_inds])} \n''')
    
    print(f'''enviro subsystem orbital energies should be ZERO (traced out):{ 
                                   np.allclose(new_orbital_energies[enviro_MO_inds],
                                                np.zeros_like(new_orbital_energies[enviro_MO_inds]))} \n''')
    
else:
    raise ValueError(f'Unknown projection method {projector_method}')


print('\nbasically want active MOs first... then enviroment')
print(f'active MO indices: {active_MO_inds} \n')
print('should appear first when orbital inds listed by energy:')
sorted([(i, new_orbital_energies[i])for i in range(new_orbital_energies.shape[0])], key=lambda x:x[1])

 <env| P |env> = - <env| F |env> : True 

active subsystem orbital energies the same: True 

enviro subsystem orbital energies should be ZERO (traced out):True 


basically want active MOs first... then enviroment
active MO indices: [0 2] 

should appear first when orbital inds listed by energy:


[(0, -9.567630085625224),
 (2, -0.30276034511375016),
 (1, -3.9785251950954056e-16),
 (3, 3.023883597375666e-16),
 (4, 1.4662095052195776e-15),
 (5, 0.4253796954062942),
 (6, 0.4253990835290896),
 (7, 0.4254439118843516),
 (8, 0.4712593200591671)]

In [964]:
print('Trace(y_ENV @ PB):', np.einsum('ij, ij', dm_enviro, projector) )
print('Trace(y_act @ PB):', np.einsum('ij, ij', dm_active, projector) )
print()
print('Trace(y_ENV @ v_emb):', np.einsum('ij, ij', dm_enviro, v_emb) )
print('Trace(y_act @ v_emb):', np.einsum('ij, ij', dm_active, v_emb) )

Trace(y_ENV @ PB): 2.37391035508439
Trace(y_act @ PB): 9.974659986866641e-18

Trace(y_ENV @ v_emb): 18.481704929202998
Trace(y_act @ v_emb): 12.856678110017615


In [968]:
# full_system_mol_EMBEDDED = gto.Mole(atom= geometry,
#                       basis=basis,
#                        charge=charge,
#                        spin=spin,
#                       )
# full_system_mol_EMBEDDED.build()

# # RE-DEFINE number of electrons in system
# full_system_mol_EMBEDDED.nelectron = 2*len(active_MO_inds) # <------ IMPORTANT!

# EMBEDDED_full_system_scf = scf.RKS(full_system_mol_EMBEDDED)
# EMBEDDED_full_system_scf.verbose=1
# EMBEDDED_full_system_scf.max_memory= memory
# EMBEDDED_full_system_scf.conv_tol = 1e-6
# EMBEDDED_full_system_scf.xc = low_level_xc_functional

# h_core = EMBEDDED_full_system_scf.get_hcore()

# # overwrite h_core to include embedding term!!!!
# EMBEDDED_full_system_scf.get_hcore = lambda *args: v_emb + h_core

# E_emb = EMBEDDED_full_system_scf.kernel()

# print(f'embedded Energy: {E_emb}')



full_system_mol_EMBEDDED = gto.Mole(atom= geometry,
                      basis=basis,
                       charge=charge,
                       spin=spin,
                      )
full_system_mol_EMBEDDED.build()

# RE-DEFINE number of electrons in system
full_system_mol_EMBEDDED.nelectron = 2*len(active_MO_inds) # <------ IMPORTANT!

EMBEDDED_full_system_scf = scf.RKS(full_system_mol_EMBEDDED)
EMBEDDED_full_system_scf.verbose=1
EMBEDDED_full_system_scf.max_memory= memory
EMBEDDED_full_system_scf.conv_tol = 1e-6
EMBEDDED_full_system_scf.xc = low_level_xc_functional



## include change of basis!
Hcore_std = EMBEDDED_full_system_scf.get_hcore()
EMBEDDED_full_system_scf.get_hcore = lambda *args: Get_new_Hcore(Hcore_std, 
                                                             Unitary_rot=V)


EMBEDDED_full_system_scf.get_veff = lambda mol=None, dm=None, dm_last=0, vhf_last=0, hermi=1: Get_new_Veff(
                                                                                    EMBEDDED_full_system_scf,
                                                                                       V,
                                                                                        dm=dm)

Hcore_rotated = EMBEDDED_full_system_scf.get_hcore()

# overwrite h_core to include embedding term!!!!
EMBEDDED_full_system_scf.get_hcore = lambda *args: v_emb + Hcore_rotated

E_emb = EMBEDDED_full_system_scf.kernel()

print(f'embedded Energy: {E_emb}')

embedded Energy: -16.015495621244828


In [969]:
print(f'''number of electrons is correct: {
                                            np.isclose(sum(EMBEDDED_full_system_scf.mo_occ),
                                            2*len(active_MO_inds))
                                          }''')

number of electrons is correct: True


In [970]:
C_active_EMBEDDED = EMBEDDED_full_system_scf.mo_coeff[:, EMBEDDED_full_system_scf.mo_occ>0]
dm_active_EMBEDDED = 2* C_active_EMBEDDED @ C_active_EMBEDDED.conj().T

# energy with STANDARD rks object!
print(test_full_system_scf.energy_elec(dm=dm_active_EMBEDDED)[0])

-42.34501786833437


In [971]:
# calculate embedding correction term

dm_correction = np.einsum('ij, ij', v_emb, dm_active_EMBEDDED-dm_active)

e_act_emb = test_full_system_scf.energy_elec(dm=dm_active_EMBEDDED,
                                        vhf= test_full_system_scf.get_veff(dm=dm_active_EMBEDDED),
                                       h1e = test_full_system_scf.get_hcore())[0]

# e_act_emb = full_system_scf.energy_elec(dm=dm_active_EMBEDDED,
#                                         vhf= full_system_scf.get_veff(dm=dm_active_EMBEDDED),
#                                        h1e = full_system_scf.get_hcore())[0]

e_mf_emb = e_act_emb + E_env + two_e_cross + test_full_system_scf.energy_nuc() + dm_correction
e_mf_emb # <-- energy from embedded DFT calc

-39.616842491819554

In [972]:
full_system_scf.e_tot

-39.61684241519855

In [973]:
print(f'''global DFT calculation == seperated calculation: {
                                                            np.isclose(e_mf_emb,
                                                                        full_system_scf.e_tot)
                                                        }''')
# expected as same functional used!

global DFT calculation == seperated calculation: True


In [974]:
full_system_mol_HIGH_LEVEL_DFT = gto.Mole(atom= geometry,
                      basis=basis,
                       charge=charge,
                       spin=spin,
                      )

full_system_mol_HIGH_LEVEL_DFT.nelectron = 2*len(active_MO_inds) # <------ IMPORTANT!
full_system_mol_HIGH_LEVEL_DFT.build()

full_system_scf_HIGH_LEVEL = scf.RKS(full_system_mol_HIGH_LEVEL_DFT)
full_system_scf_HIGH_LEVEL.verbose=1
full_system_scf_HIGH_LEVEL.max_memory= memory
full_system_scf_HIGH_LEVEL.conv_tol = 1e-6
full_system_scf_HIGH_LEVEL.xc = high_level_xc_functional # <-- BETTER functional!


# full_system_scf_HIGH_LEVEL.kernel() < --- do NOT RUN THIS


## include change of basis!
Hcore_std = full_system_scf_HIGH_LEVEL.get_hcore()
full_system_scf_HIGH_LEVEL.get_hcore = lambda *args: Get_new_Hcore(Hcore_std, 
                                                             Unitary_rot=V)


full_system_scf_HIGH_LEVEL.get_veff = lambda mol=None, dm=None, dm_last=0, vhf_last=0, hermi=1: Get_new_Veff(
                                                                                    full_system_scf_HIGH_LEVEL, #<-need this here for more expensive xc fucnctional@
                                                                                       V,
                                                                                        dm=dm)

Hcore_rotated = full_system_scf_HIGH_LEVEL.get_hcore()

# overwrite h_core to include embedding term!!!!
full_system_scf_HIGH_LEVEL.get_hcore = lambda *args: v_emb + Hcore_rotated

e_act_emb_HIGH_LVL = full_system_scf_HIGH_LEVEL.kernel()
e_act_emb_HIGH_LVL

-16.390809031041307

In [975]:
C_active_EMBEDDED_high_lvl = full_system_scf_HIGH_LEVEL.mo_coeff[:, full_system_scf_HIGH_LEVEL.mo_occ>0]
dm_active_EMBEDDED_high_lvl = 2* C_active_EMBEDDED_high_lvl @ C_active_EMBEDDED_high_lvl.conj().T

In [976]:
# calculate embedding correction term

dm_correction_high_lvl = np.einsum('ij, ij', v_emb, dm_active_EMBEDDED_high_lvl-dm_active)

e_act_emb_high_lvl = test_full_system_scf.energy_elec(dm=dm_active_EMBEDDED_high_lvl,
                                        vhf= test_full_system_scf.get_veff(dm=dm_active_EMBEDDED_high_lvl),
                                       h1e = test_full_system_scf.get_hcore())[0]

E_high_lvl_DFT = e_act_emb_high_lvl + E_env + two_e_cross + test_full_system_scf.energy_nuc() + dm_correction_high_lvl
E_high_lvl_DFT # <-- energy from embedded DFT calc

-39.62205432802092

In [977]:
print('High level DFT in DFT error:', np.abs(E_high_lvl_DFT-my_fci.e_tot))
print('LOW level DFT in DFT error:', np.abs(e_mf_emb-my_fci.e_tot))

High level DFT in DFT error: 0.1836226390900748
LOW level DFT in DFT error: 0.18883447529144348


In [ ]:
fkaisdjlfkds

In [590]:
mu*full_system_scf.mo_energy[enviro_MO_inds]

array([-581486.259046  , -302736.45692252, -302730.4359499 ])

In [591]:
new_orbital_energies[enviro_MO_inds]

array([999999.41851301, 999999.6972629 , 999999.69726892])

In [ ]:
# S_minus_half = sp.linalg.fractional_matrix_power(S_mat, -0.5)
# S_minus_plus = sp.linalg.fractional_matrix_power(S_mat, 0.5)
# test = S_minus_plus@ C_envrio
# print(test[:,0] @ test[:,0] )
# print(test[:,0] @ test[:,1])

In [ ]:
# S_minus_half = sp.linalg.fractional_matrix_power(S_mat, -0.5)
# np.allclose(S_minus_half.conj().T, S_minus_half) # Udagger = U!!!!

In [ ]:
Fock = test_full_system_scf.get_hcore() + test_full_system_scf.get_veff(dm=dm_active + dm_enviro)

S_minus_half = sp.linalg.fractional_matrix_power(S_mat, -0.5)
S_plus_half = sp.linalg.fractional_matrix_power(S_mat, 0.5)

Fock_prime = S_minus_half  @ Fock @ S_minus_half 
C_enviro_prime = S_plus_half@ C_envrio
dm_enviro_prime = 2* C_enviro_prime@C_enviro_prime.conj().T
S_prime =  S_minus_half  @ S_mat @ S_minus_half

F_gammaB_S = Fock_prime @ dm_enviro_prime @ S_prime #@ np.eye(Fock_prime.shape[0]) #S_mat
# F_gammaB_S = S_half@ F_gammaB_S  @  S_neg_half
projector = -0.5 * (F_gammaB_S + F_gammaB_S.T)
# projector =  S_neg_half @ projector @  S_half
print(np.allclose(projector @ C_enviro_prime[:,enviro_MO_inds[0]],
                  C_enviro_prime[:,enviro_MO_inds[0]]))

print(projector @ C_enviro_prime[:,0])
print(C_enviro_prime[:,0])

In [ ]:
Pb = dm_enviro_prime @ S_prime # identity

PROJEC = Fock@Pb+Pb@Fock

PROJEC

In [ ]:
print(C_enviro_prime[:,0].conj().T @ projector/2 @ C_enviro_prime[:,0])
print(C_enviro_prime[:,0].conj().T @ Fock_prime @ C_enviro_prime[:,0])

In [ ]:
# Fock_A = test_full_system_scf.get_hcore() + test_full_system_scf.get_veff(dm=dm_active)
# Fock_A_prime = S_minus_half  @ Fock_A @ S_minus_half 
# np.around(Fock_A_prime@projector - projector@Fock_A_prime, 3)

In [ ]:
F_P = projector@Fock_prime

ind=2
print(C_enviro_prime[:,ind].conj().T @F_P@C_enviro_prime[:,ind])
print(C_enviro_prime[:,ind].conj().T @Fock_prime@C_enviro_prime[:,ind])
print()
print(C_enviro_prime[:,ind].conj().T @(Fock_prime+projector)@C_enviro_prime[:,ind])
print(C_enviro_prime[:,ind].conj().T @(2*Fock_prime+projector)@C_enviro_prime[:,ind]) # 2F_full - F_b = F_a only!

In [ ]:
print(2*projector@C_enviro_prime[:,1])
print(C_enviro_prime[:,1])

In [ ]:
# projector[:,active_MO_inds] = np.zeros((Fock_prime.shape[0], len(active_MO_inds)))
# print(projector[:,active_MO_inds])

new_F = Fock_prime+projector

# ind=2
# print(C_enviro_prime[:,ind].conj().T @new_F@C_enviro_prime[:,ind])
# print(C_envrio[:,ind].conj().T @Fock@C_envrio[:,ind])

C_full_loc = S_plus_half @ C_all_localized_and_virt

print(enviro_MO_inds)
print(active_MO_inds)
print()
ind=0
# print(C_full_loc[:,ind].conj().T @new_F@C_full_loc[:,ind])
print(C_all_localized_and_virt[:,ind].conj().T @Fock@C_all_localized_and_virt[:,ind])

print(C_full_loc[:,ind].conj().T @(new_F+Fock_prime)/2@C_full_loc[:,ind])

In [ ]:
projector@C_enviro_prime[:,0]
C_enviro_prime[:,0]

# 4. Define V_embed

$$h^{A_{\text{act}} \text{ in } B_{\text{env}}} = h^{\text{core}} + v^{\text{embed}}$$

$$v^{\text{embed}} = g[\gamma^{A_{\text{act}}} + \gamma^{B_{\text{env}}} ] - g[\gamma^{A_{\text{act}}}] + P_{\text{projector}}$$

In [ ]:
# projector_method = 'huzinaga'
projector_method = 'mu_shfit'

V_embed  = vqe_in_dft.Get_embedded_potential_operator(projector_method, 
                                full_system_scf, 
                                dm_active, 
                                dm_enviro, 
                                check_Hcore_is_correct=True, 
                                mu_shift_val=1e6,
                                check_Vemb=True)

In [ ]:
projector_method = 'huzinaga'
# projector_method = 'mu_shfit'
# projector_method = 'manual'

if projector_method == 'huzinaga':
    Fock = test_full_system_scf.get_hcore() + test_full_system_scf.get_veff(dm=dm_active + dm_enviro)
    F_gammaB_S = Fock @ dm_enviro @ S_mat
    projector = -0.5 * (F_gammaB_S + F_gammaB_S.T)
elif projector_method == 'mu_shfit':
    mu = 1e6
    projector = mu * (S_mat @ dm_enviro  @ S_mat)
elif projector_method == 'manual':
    ## manual projector
    S_mat = full_system_scf.get_ovlp()
    S_half = sp.linalg.fractional_matrix_power(full_system_scf.get_ovlp() , 0.5)
    S_neg_half = sp.linalg.fractional_matrix_power(full_system_scf.get_ovlp() , -0.5)

    Loc_Ortho = S_half@ C_all_localized_and_virt

    PROJ_ortho = np.zeros_like(S_mat)
    for MO_ind in range(C_all_localized_and_virt.shape[1]):
        if MO_ind in enviro_MO_inds:
            outer = np.outer(Loc_Ortho[:, MO_ind], Loc_Ortho[:, MO_ind])
            PROJ_ortho+=outer
        else:
            continue
#     print(np.allclose(PROJ_ortho @ Loc_Ortho[:,enviro_MO_inds[0]], Loc_Ortho[:,enviro_MO_inds[0]]))
    # convert to localized basis (rather than ortho one)
    PROJ_bas = S_neg_half@ PROJ_ortho  @ S_half
    projector = PROJ_bas
#     print(np.allclose(PROJ_bas @ C_all_localized_and_virt[:,enviro_MO_inds[0]],
#                   C_all_localized_and_virt[:,enviro_MO_inds[0]]))
else:
    raise ValueError(f'Unknown projection method {projector_method}')

g_A_and_B = test_full_system_scf.get_veff(dm=dm_active+dm_enviro)

g_A = test_full_system_scf.get_veff(dm=dm_active)

v_emb = g_A_and_B - g_A + projector

In [ ]:
print('Trace(y_ENV @ PB):', np.einsum('ij, ij', dm_enviro, projector) )
print('Trace(y_act @ PB):', np.einsum('ij, ij', dm_active, projector) )
print()
print('Trace(y_ENV @ v_emb):', np.einsum('ij, ij', dm_enviro, v_emb) )
print('Trace(y_act @ v_emb):', np.einsum('ij, ij', dm_active, v_emb) )

In [ ]:
# Fock = test_full_system_scf.get_hcore() + test_full_system_scf.get_veff(dm=dm_active + dm_enviro)

# Fock_prime = S_half  @ Fock @ S_neg_half 

# F_gammaB_S = Fock_prime @ dm_enviro @ np.eye(Fock_prime.shape[0]) #S_mat
# # F_gammaB_S = S_half@ F_gammaB_S  @  S_neg_half
# projector = -0.5 * (F_gammaB_S + F_gammaB_S.T)
# projector =  S_neg_half @ projector @  S_half
# print(np.allclose(projector @ C_all_localized_and_virt[:,enviro_MO_inds[0]],
#                   C_all_localized_and_virt[:,enviro_MO_inds[0]]))

# projector @ C_all_localized_and_virt[:,1]

In [ ]:
Fock_prime@Loc_Ortho[:,0]
Fock@C_all_localized_and_virt[:,0]

In [ ]:
Loc_Ortho = S_half@ C_all_localized_and_virt #@ S_neg_half
Loc_Ortho[:,0].dot(Loc_Ortho[:,3])

Fock_prime = Fock @ S_neg_half
S_prime =  S_mat @ S_neg_half

ind=2
print(np.allclose(Fock_prime@ Loc_Ortho[:, ind],
                 Fock@ C_all_localized_and_virt[:, ind]))

PROJ_ortho = np.zeros_like(S_mat)
for MO_ind in range(C_all_localized_and_virt.shape[1]):
    if MO_ind in enviro_MO_inds:
        outer_proj = np.outer(Loc_Ortho[:, MO_ind], Loc_Ortho[:, MO_ind])
        PROJ_ortho+= outer_proj
    else:
        continue

print(np.allclose(PROJ_ortho@Loc_Ortho[:,ind] , Loc_Ortho[:,ind]))
P_I = -1 * (Fock_prime@PROJ_ortho + PROJ_ortho@Fock_prime)

ind= enviro_MO_inds[2]
print(Loc_Ortho[:,ind].conj().T @ (Fock_prime)@Loc_Ortho[:,ind])
print(Loc_Ortho[:,ind].conj().T @ (Fock_prime+P_I)@Loc_Ortho[:,ind])

P_I_std_basis = P_I@ S_half

print(Loc_Ortho[:,ind].conj().T @ S_neg_half.conj().T @ (Fock_prime@ S_half)@ S_neg_half@ Loc_Ortho[:,ind])

In [ ]:
(S_half @ C_all_localized_and_virt)

In [ ]:
Loc_Ortho = S_half@ C_all_localized_and_virt 

print(Loc_Ortho[:,0].dot(Loc_Ortho[:,1]))

# Fock_prime = S_half.conj().T @ Fock @ S_half
Fock_prime =  Fock @ S_neg_half 

print(Loc_Ortho[:,ind].conj().T @ Fock_prime@ Loc_Ortho[:,ind])
print(C_all_localized_and_virt[:,ind].conj().T @ Fock@ C_all_localized_and_virt[:,ind])

# S_prime =  S_mat @ S_neg_half

# ind=2
# print(np.allclose(Fock_prime@ Loc_Ortho[:, ind],
#                  Fock@ C_all_localized_and_virt[:, ind]))

# PROJ_ortho = np.zeros_like(S_mat)
# for MO_ind in range(C_all_localized_and_virt.shape[1]):
#     if MO_ind in enviro_MO_inds:
#         outer_proj = np.outer(Loc_Ortho[:, MO_ind], Loc_Ortho[:, MO_ind])
#         PROJ_ortho+= outer_proj
#     else:
#         continue

# print(np.allclose(PROJ_ortho@Loc_Ortho[:,ind] , Loc_Ortho[:,ind]))
# P_I = -1 * (Fock_prime@PROJ_ortho + PROJ_ortho@Fock_prime)

# ind= enviro_MO_inds[2]
# print(Loc_Ortho[:,ind].conj().T @ (Fock_prime)@Loc_Ortho[:,ind])
# print(Loc_Ortho[:,ind].conj().T @ (Fock_prime+P_I)@Loc_Ortho[:,ind])

# P_I_std_basis = P_I@ S_half

# print(Loc_Ortho[:,ind].conj().T @ S_neg_half.conj().T @ (Fock_prime@ S_half)@ S_neg_half@ Loc_Ortho[:,ind])

In [ ]:
# Loc_Ortho = S_half@ C_all_localized_and_virt #@ S_neg_half
# Loc_Ortho[:,0].dot(Loc_Ortho[:,3])

# Fock_prime = Fock @ S_neg_half
# S_prime =  S_mat @ S_neg_half

# ind=2
# print(np.allclose(Fock_prime@ Loc_Ortho[:, ind],
#                  Fock@ C_all_localized_and_virt[:, ind]))

# PROJ_ortho = np.zeros_like(S_mat)
# for MO_ind in range(C_all_localized_and_virt.shape[1]):
#     if MO_ind in enviro_MO_inds:
#         outer_proj = np.outer(C_all_localized_and_virt[:, MO_ind], C_all_localized_and_virt[:, MO_ind])
#         PROJ_ortho+= outer_proj
#     else:
#         continue

# # print(np.allclose(PROJ_ortho@Loc_Ortho[:,ind] , Loc_Ortho[:,ind]))
# P_I = -1 * (Fock@S_half@PROJ_ortho@S_half.conj().T 
#             +
#            S_half@PROJ_ortho@S_half.conj().T @ Fock @ S_neg_half)

# ind=2
# print(C_all_localized_and_virt[:,ind].conj().T @ (Fock)@C_all_localized_and_virt[:,ind])
# print(C_all_localized_and_virt[:,ind].conj().T @ (Fock+P_I)@C_all_localized_and_virt[:,ind])

In [ ]:
FP=Fock_prime@PROJ_ortho 
np.allclose(FP.T,
            PROJ_ortho@Fock_prime)

In [ ]:
# Fock = test_full_system_scf.get_hcore() + test_full_system_scf.get_veff(dm=dm_active + dm_enviro)
# projector = (S_mat @ dm_enviro  @ S_mat)
# huzinaga = -1*(C_all_localized_and_virt.conj().T @ (Fock@projector + projector@Fock) @ C_all_localized_and_virt)

# Fock_A = test_full_system_scf.get_hcore() + test_full_system_scf.get_veff(dm=dm_active)

# ind=3
# print(C_all_localized_and_virt[:,ind].conj().T @ (Fock)@C_all_localized_and_virt[:,ind])
# print(C_all_localized_and_virt[:,ind].conj().T @ (Fock+huzinaga)@C_all_localized_and_virt[:,ind])

# Fock_A@huzinaga - huzinaga@Fock_A


##########

# mu = 1e6
# projector = (S_mat @ dm_enviro  @ S_mat)

# print(projector@C_all_localized_and_virt[:,0])
# print(C_all_localized_and_virt[:,0])

S_mat = full_system_scf.get_ovlp()
S_half = sp.linalg.fractional_matrix_power(full_system_scf.get_ovlp() , 0.5)
S_neg_half = sp.linalg.fractional_matrix_power(full_system_scf.get_ovlp() , -0.5)

Loc_Ortho = S_half@ C_all_localized_and_virt

PROJ_ortho = np.zeros_like(S_mat)
for MO_ind in range(C_all_localized_and_virt.shape[1]):
    if MO_ind in enviro_MO_inds:
        outer = np.outer(Loc_Ortho[:, MO_ind], Loc_Ortho[:, MO_ind])
        PROJ_ortho+=outer
    else:
        continue
#     print(np.allclose(PROJ_ortho @ Loc_Ortho[:,enviro_MO_inds[0]], Loc_Ortho[:,enviro_MO_inds[0]]))
# convert to localized basis (rather than ortho one)
PROJ_bas = S_neg_half@ PROJ_ortho  @ S_half
projector = PROJ_bas
print(np.allclose(PROJ_bas @ C_all_localized_and_virt[:,enviro_MO_inds[1]],
              C_all_localized_and_virt[:,enviro_MO_inds[1]]))


# huzinaga = -1*(C_all_localized_and_virt.conj().T @ (Fock@projector + projector@Fock) @ C_all_localized_and_virt)
huzinaga = -(Fock@projector + projector@Fock)

ind=3
print(C_all_localized_and_virt[:,ind].conj().T @ (Fock)@C_all_localized_and_virt[:,ind])
print(C_all_localized_and_virt[:,ind].conj().T @ (huzinaga)@C_all_localized_and_virt[:,ind])
print()
print(C_all_localized_and_virt[:,ind].conj().T @ (projector)@C_all_localized_and_virt[:,ind])
print(C_all_localized_and_virt[:,ind].conj().T @ C_all_localized_and_virt[:,ind])

print()
# Proj_Fock = -1*(Fock@projector) + Fock
Proj_Fock = -1*(projector@Fock + Fock@projector) + Fock

print(C_all_localized_and_virt[:,ind].conj().T @ (Proj_Fock)@C_all_localized_and_virt[:,ind])
print(C_all_localized_and_virt[:,ind].conj().T @ (Fock)@C_all_localized_and_virt[:,ind])

In [ ]:
# PROJ_ortho_new[:,1].conj().T @ PROJ_ortho_new[:,3]

In [ ]:
print(np.allclose(PROJ_ortho@Loc_Ortho[:, 1],
           Loc_Ortho[:, 1]))

# PROJ_ortho_new = PROJ_ortho.copy()
# PROJ_ortho_new[:, active_MO_inds] = np.zeros_like(C_active)

S_minus_half = sp.linalg.fractional_matrix_power(S_mat, -0.5)
S_plus_half = sp.linalg.fractional_matrix_power(S_mat, 0.5)

Fock_prime = S_minus_half@Fock @ S_minus_half
AA=Loc_Ortho.conj().T @ (Fock_prime)@Loc_Ortho
# print(np.around(AA,2 ))

HUZ =  -(Fock_prime@PROJ_ortho + PROJ_ortho@Fock_prime)/2

BB = Loc_Ortho.conj().T @ (Fock_prime+HUZ)@Loc_Ortho
print(np.around(BB,2))

print()
print(np.around(np.diag(AA-BB),2))
print(np.around(np.diag(AA),2))
print(np.around(np.diag(BB),2))

print(np.around(Loc_Ortho[:, enviro_MO_inds].conj().T @ (HUZ)@Loc_Ortho[:, enviro_MO_inds], 3))
print(np.around(Loc_Ortho[:, enviro_MO_inds].conj().T @ (Fock_prime)@Loc_Ortho[:, enviro_MO_inds], 3))
# Loc_Ortho[:, enviro_MO_inds].conj().T @ (Fock_prime)@Loc_Ortho[:, enviro_MO_inds]

In [ ]:
S_mat = full_system_scf.get_ovlp()
S_half = sp.linalg.fractional_matrix_power(full_system_scf.get_ovlp() , 0.5)
S_neg_half = sp.linalg.fractional_matrix_power(full_system_scf.get_ovlp() , -0.5)

Loc_Ortho = S_half@ C_all_localized_and_virt

PROJ_ortho = np.zeros_like(S_mat)
for MO_ind in range(C_all_localized_and_virt.shape[1]):
    if MO_ind in enviro_MO_inds:
        outer = np.outer(Loc_Ortho[:, MO_ind], Loc_Ortho[:, MO_ind])
        PROJ_ortho+=outer
    else:
        continue
#     print(np.allclose(PROJ_ortho @ Loc_Ortho[:,enviro_MO_inds[0]], Loc_Ortho[:,enviro_MO_inds[0]]))
# convert to localized basis (rather than ortho one)



print(np.allclose(PROJ_ortho@Loc_Ortho[:, 1],
           Loc_Ortho[:, 1]))

# PROJ_ortho_new = PROJ_ortho.copy()
# PROJ_ortho_new[:, active_MO_inds] = np.zeros_like(C_active)


Fock_prime = S_neg_half@Fock @ S_neg_half
AA=Loc_Ortho.conj().T @ (Fock_prime)@Loc_Ortho
# print(np.around(AA,2 ))

HUZ =  -(Fock_prime@PROJ_ortho + PROJ_ortho@Fock_prime)/2

BB = Loc_Ortho.conj().T @ (Fock_prime+HUZ)@Loc_Ortho
print(np.around(BB,2))

print()
print(np.around(np.diag(AA-BB),2))
print(np.around(np.diag(AA),2))
print(np.around(np.diag(BB),2))

print(np.around(Loc_Ortho[:, enviro_MO_inds].conj().T @ (HUZ)@Loc_Ortho[:, enviro_MO_inds], 3))
print(np.around(Loc_Ortho[:, enviro_MO_inds].conj().T @ (Fock_prime)@Loc_Ortho[:, enviro_MO_inds], 3))

HUZ_standard_Basis = S_half@ HUZ  @ S_half

print(np.around(C_all_localized_and_virt[:, enviro_MO_inds].conj().T @ (HUZ_standard_Basis)@C_all_localized_and_virt[:, enviro_MO_inds], 3))
print(np.around(C_all_localized_and_virt.conj().T @ (Fock+HUZ_standard_Basis)@C_all_localized_and_virt, 3))

In [ ]:
C_env_TEST = np.zeros_like(C_all_localized_and_virt) # zero out system A
C_env_TEST[:,enviro_MO_inds] = C_envrio # only include system B


np.around(C_env_TEST.conj().T @ -(Fock) @C_env_TEST, 5)

PPP = -0.125*(C_env_TEST.conj().T @ (Fock) @C_env_TEST)

ind=1
print(C_all_localized_and_virt[:,ind].conj().T @ (Fock)@C_all_localized_and_virt[:,ind])
print(C_all_localized_and_virt[:,ind].conj().T @ (Fock+PPP)@C_all_localized_and_virt[:,ind])

In [ ]:
Loc_Ortho[:, MO_ind].T

In [ ]:
outer = np.outer(Loc_Ortho[:, MO_ind], Loc_Ortho[:, MO_ind])
outer2  = Loc_Ortho[:, MO_ind].reshape([len(Loc_Ortho[:, MO_ind]),1])@ Loc_Ortho[:, MO_ind].reshape([1, len(Loc_Ortho[:, MO_ind])])
np.allclose(outer, outer2)

In [ ]:
np.trace(S_mat@dm_active)

In [ ]:
test_1= -0.5*(Fock@ dm_enviro@ S_mat + S_mat@dm_enviro@Fock)
# test_1= -0.25*(Fock@ dm_enviro@ S_mat + S_mat@dm_enviro@Fock) # 0.25 will remove terms! We want to PUSH to higher E!

np.diag(np.around(C_all_localized_and_virt.conj().T @ (test_1+Fock) @C_all_localized_and_virt,3))
# np.around(C_all_localized_and_virt.conj().T @ (test_1+Fock) @C_all_localized_and_virt,3)
# np.around(C_all_localized_and_virt.conj().T @ Fock @C_all_localized_and_virt,3)

In [ ]:
## manual projector

## 1. convert to orthogonal C_matrix
S_mat = full_system_scf.get_ovlp()
S_half = sp.linalg.fractional_matrix_power(full_system_scf.get_ovlp() , 0.5)
S_neg_half = sp.linalg.fractional_matrix_power(full_system_scf.get_ovlp() , -0.5)

Loc_Ortho = S_half@ C_all_localized_and_virt

## 2. Define projector that projects MO orbs of subsystem B onto themselves and system A onto zero state!
##### (do this in orthongoal C_matrix!)
PROJ_ortho = np.zeros_like(S_mat)
for MO_ind in range(C_all_localized_and_virt.shape[1]):
    if MO_ind in enviro_MO_inds:
        outer = np.outer(Loc_Ortho[:, MO_ind], Loc_Ortho[:, MO_ind])
        PROJ_ortho+=outer
    else:
        continue

print(f'''Are subsystem B (env) projected onto themselves: {
        np.allclose(PROJ_ortho@Loc_Ortho[:, enviro_MO_inds], 
        Loc_Ortho[:, enviro_MO_inds])}''') # projected onto itself

print(f'''Are subsystem A traced out?: {
        np.allclose(PROJ_ortho@Loc_Ortho[:, active_MO_inds], 
        np.zeros_like(Loc_Ortho[:, active_MO_inds]))}''') # # projected onto zeros!
        
        
#### 3. NEXT USING PROJECTOR in ortho basis define projector via different methods
        
        
projector_method = 'huzinaga'
# projector_method = 'mu_shfit'

if projector_method == 'huzinaga':
    Fock = test_full_system_scf.get_hcore() + test_full_system_scf.get_veff(dm=dm_active + dm_enviro)
    Fock_ortho = S_neg_half@Fock @ S_neg_half
    
    projector_ortho =  -0.5*(Fock_ortho@PROJ_ortho + PROJ_ortho@Fock_ortho) # 0.5 for restricted calc!
    
    projector = S_half @ projector_ortho  @ S_half
    
elif projector_method == 'mu_shfit':
    mu = 1e6
    projector_ortho = mu * PROJ_ortho
    
#     projector = S_neg_half @ projector_ortho  @ S_half
    projector = S_half @ projector_ortho  @ S_half
else:
    raise ValueError(f'Unknown projection method {projector_method}')

##### 4. Define projector in standard (non-orthogonal basis)
# projector = S_half @ projector_ortho  @ S_half


### 5. now can build relevant ops

g_A_and_B = test_full_system_scf.get_veff(dm=dm_active+dm_enviro)

g_A = test_full_system_scf.get_veff(dm=dm_active)

v_emb = g_A_and_B - g_A + projector

In [ ]:
print(np.around(C_all_localized_and_virt[:, enviro_MO_inds].conj().T @projector@C_all_localized_and_virt[:, enviro_MO_inds],3))

print(np.around(C_all_localized_and_virt[:, enviro_MO_inds].conj().T @Fock@C_all_localized_and_virt[:, enviro_MO_inds],3))


In [ ]:
np.allclose(projector@C_all_localized_and_virt[:, enviro_MO_inds],
           C_all_localized_and_virt[:, enviro_MO_inds])

In [ ]:
projector@C_all_localized_and_virt[:, enviro_MO_inds]
# (Fock)@C_all_localized_and_virt[:, enviro_MO_inds]
(Fock+projector)@C_all_localized_and_virt[:, enviro_MO_inds]

print(np.around(C_all_localized_and_virt.conj().T @ (Fock+projector)@C_all_localized_and_virt, 2))
print()
print(np.around(C_all_localized_and_virt.conj().T @ (Fock)@C_all_localized_and_virt, 2))
print(np.around(C_all_localized_and_virt.conj().T @ (projector)@C_all_localized_and_virt, 2))

In [ ]:
print(f'''Are subsystem A projected onto zeros: {
    np.allclose(projector@C_all_localized_and_virt[:, active_MO_inds], 
            np.zeros_like(C_all_localized_and_virt[:, active_MO_inds]),7)}''') 


# np.zeros_like(C_all_localized_and_virt[:, active_MO_inds])

np.allclose(np.around(projector@C_all_localized_and_virt[:, active_MO_inds],7),
           np.zeros_like(C_all_localized_and_virt[:, active_MO_inds]))

In [ ]:
if projector_method == 'huzinaga':
        print(f'''Are subsystem B (env) projected to -E of Fock subsystem B: {
            np.allclose(Fock@C_all_localized_and_virt[:, enviro_MO_inds],
                        -1*projector@C_all_localized_and_virt[:, enviro_MO_inds])}''') # projected onto neg E orbs
        
        print(f'''Are subsystem A projected onto zeros: {
                np.allclose(np.around(projector@C_all_localized_and_virt[:, active_MO_inds],7), 
                np.zeros_like(C_all_localized_and_virt[:, active_MO_inds]))}''') # projected out
        
elif projector_method == 'mu_shfit':
    print(f'''Are subsystem B (env) projected to high E: {
            np.allclose(projector@C_all_localized_and_virt[:, enviro_MO_inds], 
            mu*C_all_localized_and_virt[:, enviro_MO_inds])}''') # projected onto increased E orbs
    
    print(f'''Are subsystem A projected onto zeros: {
            np.allclose(projector@C_all_localized_and_virt[:, active_MO_inds], 
            np.zeros_like(C_all_localized_and_virt[:, active_MO_inds]))}''') # projected out


In [ ]:
POL = S_neg_half@ PROJ_ortho  @ S_half
# POL = S_half@ PROJ_ortho  @ S_half
print(f'''Are subsystem B (env) projected onto themselves: {
        np.allclose(POL@C_all_localized_and_virt[:, enviro_MO_inds], 
        C_all_localized_and_virt[:, enviro_MO_inds])}''') # projected onto itself

print(f'''Are subsystem A projected onto zeros: {
        np.allclose(POL@C_all_localized_and_virt[:, active_MO_inds], 
        np.zeros_like(C_all_localized_and_virt[:, active_MO_inds]))}''') # projected out

In [ ]:
projector@C_all_localized_and_virt[:, active_MO_inds]
# C_all_localized_and_virt[:, active_MO_inds]

In [ ]:
print(f'''Are subsystem B (env) projected onto themselves: {
        np.allclose(projector@C_all_localized_and_virt[:, enviro_MO_inds], 
        C_all_localized_and_virt[:, enviro_MO_inds])}''') # projected onto itself

print(f'''Are subsystem A traced out?: {
        np.allclose(PROJ_ortho@Loc_Ortho[:, active_MO_inds], 
        np.zeros_like(Loc_Ortho[:, active_MO_inds]))}''') # # projected onto zeros!

In [ ]:
print('Trace(y_ENV @ PB):', np.einsum('ij, ij', dm_enviro, projector) )
print('Trace(y_act @ PB):', np.einsum('ij, ij', dm_active, projector) )
print()
print('Trace(y_ENV @ v_emb):', np.einsum('ij, ij', dm_enviro, v_emb) )
print('Trace(y_act @ v_emb):', np.einsum('ij, ij', dm_active, v_emb) )
print('Trace(y_act @ v_emb):', np.einsum('ij, ij', dm_active, g_A_and_B - g_A ) )

In [ ]:
Fock_ortho = S_neg_half@Fock @ S_neg_half
# print(np.around(Loc_Ortho.conj().T@(projector_ortho+Fock_ortho)@Loc_Ortho,3))
np.diag(np.around(Loc_Ortho.conj().T@(projector_ortho+Fock_ortho)@Loc_Ortho,3))

In [ ]:
PROJ_bas @ C_all_localized_and_virt[:,2]

In [ ]:
print(np.allclose(PROJ_bas @ C_all_localized_and_virt[:,0],
              C_all_localized_and_virt[:,0]))

In [ ]:
## manual projector
S_mat = full_system_scf.get_ovlp()
S_half = sp.linalg.fractional_matrix_power(full_system_scf.get_ovlp() , 0.5)

Loc_Ortho = S_half@ C_all_localized_and_virt
Fock = test_full_system_scf.get_hcore() + test_full_system_scf.get_veff(dm=dm_active + dm_enviro)
# Fock_prime = S_neg_half@ Fock @ S_half
Fock_prime = S_half.conj().T @ Fock @ S_half

np.allclose(Fock_prime@Loc_Ortho[:,0], Fock@C_all_localized_and_virt[:,0])
# PROJ_ortho = np.zeros_like(S_mat)
# for MO_ind in range(C_all_localized_and_virt.shape[1]):
#     if MO_ind in enviro_MO_inds:
#         outer = np.outer(Loc_Ortho[:, MO_ind], Loc_Ortho[:, MO_ind])
#         PROJ_ortho+=outer
#     else:
#         continue

# print(np.allclose(PROJ_ortho @ Loc_Ortho[:,enviro_MO_inds[0]], Loc_Ortho[:,enviro_MO_inds[0]]))

# # convert to localized basis (rather than ortho one)
# PROJ_bas = S_neg_half@ PROJ_ortho  @ S_half
# print(np.allclose(PROJ_bas @ C_all_localized_and_virt[:,enviro_MO_inds[0]],
#                   C_all_localized_and_virt[:,enviro_MO_inds[0]]))

# g_A_and_B = test_full_system_scf.get_veff(dm=dm_active+dm_enviro)

# g_A = test_full_system_scf.get_veff(dm=dm_active)

# projector = PROJ_bas
# v_emb = g_A_and_B - g_A + PROJ_bas


# #### Notes #####

# # U C_loc_ortho = C_std_ortho
# ### as:
#     # C_loc_ortho = S_half @ C_loc
#     # C_std_ortho = S_half @ C_std

# # can write as:
# # U (S_half @ C_loc) = (S_half @ C_std)

# # Now want to get rid of S_half on RHS:
# # S_neg_half @ U S_half @ C_loc = (S_neg_half @ S_half @ C_std)
# # (S_neg_half @ U S_half) @ C_loc = C_std
# # U_new @ C_loc = C_std
# ### where:
#     # U_new = S_neg_half @ U S_half
    
# #### Notes #####

# U_new = S_neg_half@ U  @ S_half
# np.allclose(U_new@C_all_localized,
#             full_system_scf.mo_coeff[:,full_system_scf.mo_occ>0])


In [ ]:
# ## manual projector
# S_mat = full_system_scf.get_ovlp()
# S_half = sp.linalg.fractional_matrix_power(full_system_scf.get_ovlp() , 0.5)

# Loc_Ortho = S_half@ C_all_localized_and_virt

# PROJ_ortho = np.zeros_like(S_mat)
# for MO_ind in range(C_all_localized_and_virt.shape[1]):
#     if MO_ind in enviro_MO_inds:
#         outer = np.outer(Loc_Ortho[:, MO_ind], Loc_Ortho[:, MO_ind])
#         PROJ_ortho+=outer
#     else:
#         continue

# print(np.allclose(PROJ_ortho @ Loc_Ortho[:,enviro_MO_inds[0]], Loc_Ortho[:,enviro_MO_inds[0]]))

# # convert to localized basis (rather than ortho one)
# PROJ_bas = S_neg_half@ PROJ_ortho  @ S_half
# print(np.allclose(PROJ_bas @ C_all_localized_and_virt[:,enviro_MO_inds[0]],
#                   C_all_localized_and_virt[:,enviro_MO_inds[0]]))

# g_A_and_B = test_full_system_scf.get_veff(dm=dm_active+dm_enviro)

# g_A = test_full_system_scf.get_veff(dm=dm_active)

# projector = PROJ_bas
# v_emb = g_A_and_B - g_A + PROJ_bas


In [ ]:
C_loc_test = test_full_system_scf.mo_coeff[:,test_full_system_scf.mo_occ>0]

IND=3
# print(C_loc_test[:,IND].conj().T @ (v_emb + Fock) @ C_loc_test[:,IND])
print(C_loc_test[:,IND].conj().T @ (projector) @ C_loc_test[:,IND])

In [ ]:
Fock_A = test_full_system_scf.get_hcore() + test_full_system_scf.get_veff(dm=dm_active)
print(np.around(Fock_A@projector - projector@Fock_A, 1))
print()

# diagonal entries are zero!
np.around(np.diag((Fock_A@projector - projector@Fock_A)))

In [ ]:
n_docc

In [ ]:
n_docc = int(np.around(np.trace(dm_active @ S_mat),2)//2)

### consts
g_A_and_B = test_full_system_scf.get_veff(dm=dm_active+dm_enviro)
g_A = test_full_system_scf.get_veff(dm=dm_active)
H_core_std = H_core_standard = scf.hf.get_hcore(test_full_system_scf.mol)
###

Huz_PROJECTOR= -0.5*(H_core_std@ dm_enviro@ S_mat + S_mat@dm_enviro@H_core_std)
F_guess = H_core_std + Huz_PROJECTOR

X = sp.linalg.fractional_matrix_power(S_mat, -0.5)
eri_ao = full_system_mol.intor('int2e') 

F_prime = X.conj().T @ F_guess @ X

epsilon, C_prime = np.linalg.eigh(F_prime)

# transofrm C_prime back to AO basis
C = X @ C_prime

# get occupied orbitals
C_occ = C[:, :n_docc]

# build density matrix from occupied orbitals!
D_active_emb = C_occ @ C_occ.T
# D = np.einsum('pi,qi->pq', C_occ, C_occ, optimize=True)


#alg
max_iter = 100
HF_energy =0
E_previous = 0
E_tol = 1e-6

for i in range(max_iter+1):
    
    # build Veff for Y_embedded_A
    J_mat = np.einsum('mvls,ls -> mv', eri_ao, D_active_emb, optimize=True)
    K_mat = np.einsum('mlvs,ls -> mv', eri_ao, D_active_emb, optimize=True)
    
    g_A_emb = 2*J_mat - K_mat
    
    
    Huz_PROJECTOR= -0.5*(Fock@ dm_enviro@ S_mat + S_mat@dm_enviro@Fock)
    H_core_emb = H_core_std + g_A_and_B - g_A  + Huz_PROJECTOR 
    
    Fock = H_core_emb + g_A_emb
    
    
    ### find RHF energy
    HF_energy = np.einsum('pq,pq ->', D_active_emb, (Fock+H_core_std), optimize=True) + full_system_mol.energy_nuc()
    print(HF_energy)
    #     HF_energy =np.trace(H_core @ D) + np.trace(Fock @ D) + full_system_mol.energy_nuc()
    # (no 0.5 here as included in D mat natively!)
    
    ### check convergence
    if np.abs(HF_energy-E_previous)<E_tol:
        break
        
        
    # if not convereged store old result
    E_previous = HF_energy
    
    ## compute new orbital guess
    F_prime = X.conj().T @ Fock @ X

    epsilon, C_prime = np.linalg.eigh(F_prime)

    # transofrm C_prime back to AO basis
    C = X @ C_prime

    # get occupied orbitals
    C_occ = C[:, :n_docc]

    # build density matrix from occupied orbitals!
#     D = C_occ @ C_occ.T
    D_active_emb = np.einsum('pi,qi->pq', C_occ, C_occ, optimize=True)
    
    if i==max_iter:
        raise ValueError('Maximum number of SCF iterations exceeded')
        
print(f'final RHF SCF energy {HF_energy}')

In [ ]:
full_system_mol_EMBEDDED = gto.Mole(atom= geometry,
                      basis=basis,
                       charge=charge,
                       spin=spin,
                      )
full_system_mol_EMBEDDED.build()

# RE-DEFINE number of electrons in system
full_system_mol_EMBEDDED.nelectron = 2*len(active_MO_inds) # <------ IMPORTANT!

EMBEDDED_full_system_scf = scf.RKS(full_system_mol_EMBEDDED)
EMBEDDED_full_system_scf.verbose=1
EMBEDDED_full_system_scf.max_memory= memory
EMBEDDED_full_system_scf.conv_tol = 1e-6
EMBEDDED_full_system_scf.xc = low_level_xc_functional

h_core = EMBEDDED_full_system_scf.get_hcore()

# overwrite h_core to include embedding term!!!!
EMBEDDED_full_system_scf.get_hcore = lambda *args: v_emb + h_core

E_emb = EMBEDDED_full_system_scf.kernel()

print(f'embedded Energy: {E_emb}')

In [ ]:
EMBEDDED_full_system_scf.mo_energy

In [ ]:
old_calc = EMBEDDED_full_system_scf.mo_energy

In [ ]:
print(np.around(old_calc,3 ))
print(np.around(EMBEDDED_full_system_scf.mo_energy,3 ))

In [ ]:
np.allclose(EMBEDDED_full_system_scf.mo_energy[EMBEDDED_full_system_scf.mo_occ>0],
            old_calc[EMBEDDED_full_system_scf.mo_occ>0])
print(old_calc[EMBEDDED_full_system_scf.mo_occ>0])
print(EMBEDDED_full_system_scf.mo_energy[EMBEDDED_full_system_scf.mo_occ>0])

In [ ]:
# active_C = np.zeros_like(Fock)
# active_C[:, active_MO_inds] = C_active
# print(np.diag(active_C.conj().T @ Fock @ active_C))
# print()

# enviro_C = np.zeros_like(Fock)
# enviro_C[:, enviro_MO_inds] =C_envrio
# print(np.diag(enviro_C.conj().T @ Fock @ enviro_C))

# Fock_A = test_full_system_scf.get_hcore() + test_full_system_scf.get_veff(dm=dm_active)

# Pro_new = active_C.conj().T @ projector @ active_C
# print(np.around(Pro_new@Fock_A, 2))

# Pro_new = enviro_C.conj().T @ projector @ enviro_C
# print(np.around(Pro_new@Fock_A, 2))

In [ ]:
# active_C = np.zeros_like(Fock)
# active_C[:, active_MO_inds] = C_active
# enviro_C = np.zeros_like(Fock)
# enviro_C[:, enviro_MO_inds] =C_envrio

# F_AB = active_C.conj().T @ Fock @ enviro_C
# F_gammaB_S = F_AB @ dm_enviro @ S_mat
# projector = -0.5 * (F_gammaB_S + F_gammaB_S.T)
# Fock_A = test_full_system_scf.get_hcore() + test_full_system_scf.get_veff(dm=dm_active)
# print(np.around(Fock_A@projector - projector@Fock_A, 4))

In [ ]:
# ## manual projector
# S_mat = full_system_scf.get_ovlp()
# S_half = sp.linalg.fractional_matrix_power(full_system_scf.get_ovlp() , 0.5)

# Loc_Ortho = S_half@ C_all_localized

# PROJ_ortho = np.zeros_like(S_mat)
# for MO_ind in range(C_all_localized.shape[1]):
#     if MO_ind in enviro_MO_inds:
#         outer = np.outer(Loc_Ortho[:, MO_ind], Loc_Ortho[:, MO_ind])
#         PROJ_ortho+=outer
#     else:
#         continue

# print(np.allclose(PROJ_ortho @ Loc_Ortho[:,enviro_MO_inds[0]], Loc_Ortho[:,enviro_MO_inds[0]]))

# # convert to localized basis (rather than ortho one)
# PROJ_bas = S_neg_half@ PROJ_ortho  @ S_half
# print(np.allclose(PROJ_bas @ C_all_localized[:,enviro_MO_inds[0]], C_all_localized[:,enviro_MO_inds[0]]))

# # C_all_localized_and_virt # contains loc and virtual orbs!

In [ ]:
orbital_ind=2
np.allclose(projector @ C_loc_test[:, orbital_ind], C_loc_test[:, orbital_ind])

In [ ]:
IND=4
# print(C_loc_test[:,IND].conj().T @ (v_emb + Fock) @ C_loc_test[:,IND])
print(C_loc_test[:,IND].conj().T @ (projector) @ C_loc_test[:,IND])

In [ ]:
full_system_scf.mo_coeff[:,full_system_scf.mo_occ>0]

In [ ]:
IND=2
print(full_system_scf.mo_coeff[:,IND].conj().T @ full_system_scf.get_fock() @ full_system_scf.mo_coeff[:,IND])
print(full_system_scf.mo_energy[IND])

In [ ]:
IND=4
print(C_all_localized[:,IND].conj().T @ full_system_scf.get_fock() @ C_all_localized[:,IND])
print(full_system_scf.mo_energy[IND])

In [ ]:
U = full_system_scf.mo_coeff @ np.linalg.pinv(C_all_localized)# c
np.allclose(U @C_all_localized,
            full_system_scf.mo_coeff) # checking U@ C_loc = C_standard

In [ ]:
U1, singular_values1, rotation_matrix = np.linalg.svd(full_system_scf.get_fock(dm = dm_active + dm_enviro)
                                                      , full_matrices=True)

U2, singular_values2, rotation_matrix2 = np.linalg.svd(full_system_scf.get_fock()
                                                      , full_matrices=True)

singular_values1

In [ ]:
U1, singular_values1, rotation_matrix = np.linalg.svd(C_all_localized[:,full_system_scf.mo_occ>0], full_matrices=True)
# U2, singular_values2, V2 = np.linalg.svd(full_system_scf.mo_coeff, full_matrices=True)

C_active = C_all_localized[:,full_system_scf.mo_occ>0] @ rotation_matrix.T[:, active_MO_inds]
C_envrio = C_all_localized[:,full_system_scf.mo_occ>0] @ rotation_matrix.T[:, enviro_MO_inds]


C_matrix_all_localized_orbitals = C_all_localized[:,full_system_scf.mo_occ>0] @ rotation_matrix.T


In [ ]:
dm_localised_full_system = 2* C_matrix_all_localized_orbitals@ C_matrix_all_localized_orbitals.conj().T
dm_active =  2 * C_active @ C_active.T
dm_enviro =  2 * C_envrio @ C_envrio.T
    
bool_density_flag = np.allclose(dm_localised_full_system, dm_active + dm_enviro)
bool_density_flag

In [ ]:
V_dag, singular_values, V = np.linalg.svd(U, full_matrices=True)

print(C_all_localized[:,IND].conj().T @ (V@full_system_scf.get_fock() @ V_dag)/2 @ C_all_localized[:,IND])
print(full_system_scf.mo_energy[IND])

In [ ]:
IND=2
print(C_all_localized[:,IND].conj().T @ (U.conj().T@full_system_scf.get_fock() @ U) @ C_all_localized[:,IND])
print(full_system_scf.mo_energy[IND])

In [ ]:
eig_vals, eig_vecs = np.linalg.eigh((U.conj().T@full_system_scf.get_fock() @ U))

eig_vals2, eig_vecs2 = np.linalg.eigh(full_system_scf.get_fock())

print(eig_vals)
print(eig_vals2)

In [ ]:
dm_loc = 2 * C_all_localized[:,full_system_scf.mo_occ>0] @ C_all_localized[:,full_system_scf.mo_occ>0].conj().T

C_all_localized[:,IND].conj().T @ full_system_scf.get_fock() @ C_all_localized[:,IND]

In [ ]:
C_opt_std = full_system_scf.mo_coeff[:,full_system_scf.mo_occ>0]
dm_std = 2*C_opt_std @ C_opt_std.conj().T
np.allclose(dm_std, dm_active+dm_enviro)

In [ ]:
dm_active + dm_ac
np.allclose(full_system_scf.get_veff(dm=dm_active + dm_enviro), 
            full_system_scf.get_veff())

In [ ]:
Fock = full_system_scf.get_hcore() + full_system_scf.get_veff(dm = dm_active+ dm_enviro)
eig_vals, eig_vecs = np.linalg.eigh(Fock)

eig_vals

In [ ]:
full_system_scf.mo_energy

In [ ]:
fasdfds

In [ ]:
Fock@full_system_scf.mo

In [ ]:
(U.conj().T@Fock@U).shape

In [ ]:
U = C_all_localized @ np.linalg.pinv(full_system_scf.mo_coeff)# c

np.allclose(U @ full_system_scf.mo_coeff,
C_all_localized)

In [ ]:
# ind = 2
# print(full_system_scf.mo_coeff[:,ind].conj().T @ full_system_scf.get_fock() @ full_system_scf.mo_coeff[:,ind])
# print(full_system_scf.mo_energy[ind]) 

# F_mo = C_all_localized.conj().T @ (U.conj().T@ full_system_scf.get_fock() @U) @ C_all_localized
# np.around(F_mo, 1)

In [ ]:
C_all_localized.conj().T @ full_system_scf.get_fock() @ C_all_localized

In [ ]:
Fock = full_system_scf.get_hcore() + full_system_scf.get_veff(dm = dm_active+ dm_enviro)
F_rot = C_all_localized.conj().T @ (U.conj().T@ Fock @U) @ C_all_localized

# F_MO = full_system_scf.mo_coeff.conj().T @ Fock @ full_system_scf.mo_coeff
print(np.around(F_rot, 1))

In [ ]:
AO_slice_matrix = full_system_scf.mol.aoslice_by_atom()
ao_active_inds = np.arange(AO_slice_matrix[0,2], AO_slice_matrix[N_active_atoms-1,3])

ao_ENViro_inds = np.arange(AO_slice_matrix[N_active_atoms,2], AO_slice_matrix[-1,-1])

print(ao_active_inds)
print(ao_ENViro_inds)

In [ ]:
ao_ENViro_inds

In [ ]:
Fock = full_system_scf.get_hcore() + full_system_scf.get_veff(dm = dm_enviro)
S_mat = full_system_scf.get_ovlp()

P = np.zeros_like(Fock)
# for i in ao_ENViro_inds:
for mu in ao_ENViro_inds:
    if mu in ao_ENViro_inds:
        for nu in ao_ENViro_inds:
            if nu in ao_ENViro_inds:
                P[mu, nu]+= dm_enviro[mu, nu] * (Fock[mu, mu]*S_mat[nu, nu] + S_mat[mu, mu]*Fock[nu, nu])
    
P.shape

In [ ]:
# orbital_ind=4
# psi_MO = EMBEDDED_full_system_scf.mo_coeff[:, orbital_ind]
# E_MO =EMBEDDED_full_system_scf.mo_energy[orbital_ind]

# Fock = EMBEDDED_full_system_scf.get_fock()
# np.allclose(psi_MO.conj().T @Fock@psi_MO, E_MO)

# Fock = full_system_scf.get_hcore() + full_system_scf.get_veff(dm = dm_active+ dm_enviro)
# projector = np.zeros_like(Fock)
# for orbital_ind in range(full_system_scf.mo_coeff.shape[1]):
#     psi_MO = EMBEDDED_full_system_scf.mo_coeff[:, orbital_ind]
#     if orbital_ind in active_MO_inds:
#         outer = np.outer(psi_MO, psi_MO)
#         projector-= Fock@outer + outer@Fock
#     else:
#        # inactive 
#         continue

Fock = full_system_scf.get_hcore() + full_system_scf.get_veff(dm = dm_active+ dm_enviro)
projector = np.zeros_like(Fock)
for orbital_ind in range(C_all_localized.shape[1]):
    psi_MO = C_all_localized[:, orbital_ind]
    if orbital_ind in active_MO_inds:
        outer = np.outer(psi_MO, psi_MO)
        projector-= Fock@outer + outer@Fock
    else:
       # inactive 
        continue

In [ ]:
# Fock = full_system_scf.get_hcore() + full_system_scf.get_veff(dm = dm_active+ dm_enviro)
# S_mat = full_system_scf.get_ovlp()

# projector = np.zeros_like(Fock)
# for mu in range(Fock.shape[0]):
#     for nu in range(Fock.shape[1]):
#         projector[mu, nu] = -0.5* (dm_enviro[mu, nu]) * (Fock[mu,mu]*S_mat[nu, nu] + S_mat[mu,mu]*Fock[nu, nu])


In [ ]:
# PP = -0.5 * dm_enviro @ (Fock@S_mat+ S_mat@Fock)
# # np.allclose(projector, PP)

In [ ]:
Fock = full_system_scf.get_hcore() + full_system_scf.get_veff(dm = dm_active+ dm_enviro)
projector = np.zeros_like(Fock)
for orbital_ind in range(C_all_localized.shape[1]):
    psi_MO = C_all_localized[:, orbital_ind]
    if orbital_ind in active_MO_inds:
        outer = np.outer(psi_MO, psi_MO)
        projector = outer
    else:
       # inactive 
        continue

In [ ]:
Fock = full_system_scf.get_hcore() + full_system_scf.get_veff(dm = dm_active+ dm_enviro)
projector = np.zeros_like(Fock)
for orbital_ind in range(C_all_localized.shape[1]):
    psi_MO = C_all_localized[:, orbital_ind]
    if orbital_ind in enviro_MO_inds: # <---- ENV
        outer = np.outer(psi_MO, psi_MO)
        projector = Fock@outer + Fock@outer
    else:
       # inactive 
        continue

In [ ]:
C_all_localized[:, orbital_ind]

In [ ]:
orbital_ind=1

mo_E = full_system_scf.mo_energy[orbital_ind]

( (Fock) @ C_all_localized[:, orbital_ind])/ (C_all_localized[:, orbital_ind].conj().T@ Fock@ C_all_localized[:, orbital_ind])


In [ ]:
# F_MO =  C_all_localized.conj().T @ Fock @ C_all_localized
F_MO = full_system_scf.mo_coeff.conj().T @ Fock @ full_system_scf.mo_coeff
print(np.around(F_MO, 2))


In [ ]:
Proj_MO = np.zeros_like(F_MO)
# Proj_MO[ao_active_inds, ao_active_inds]=1
Proj_MO[ao_ENViro_inds, ao_ENViro_inds]= -1* F_MO[ao_ENViro_inds, ao_ENViro_inds]


Proj_AO = (full_system_scf.mo_coeff.conj().T@ Proj_MO 
           @ np.linalg.pinv(full_system_scf.mo_coeff))

# Proj_AO

In [ ]:
MO_ind=6
print(np.eye(Proj_MO.shape[0])[:,MO_ind] @ Proj_MO @ np.eye(Proj_MO.shape[0])[:,MO_ind])
print(np.eye(Proj_MO.shape[0])[:,MO_ind] @ F_MO @ np.eye(Proj_MO.shape[0])[:,MO_ind])
print()
print(np.eye(Proj_MO.shape[0])[:,MO_ind] @ (F_MO+Proj_MO) @ np.eye(Proj_MO.shape[0])[:,MO_ind])

In [ ]:
C_inv = np.linalg.pinv(full_system_scf.mo_coeff)
projected_fock_AO = (C_inv.conj().T@ (Proj_MO+F_MO)@ C_inv)

In [ ]:
ind = 6
full_system_scf.mo_coeff[:,ind].conj().T @ projected_fock_AO @ full_system_scf.mo_coeff[:,ind]

In [ ]:
full_system_scf.mo_coeff[:,ind].conj().T @ Fock @ full_system_scf.mo_coeff[:,ind]

In [ ]:
full_system_scf.mo_coeff[:,1]

In [ ]:
Proj_AO@full_system_scf.mo_coeff[:,1]

In [ ]:
np.allclose(np.linalg.pinv(full_system_scf.mo_coeff.conj().T)@ F_MO @ np.linalg.pinv(full_system_scf.mo_coeff),
           Fock)

In [ ]:
full_system_scf.mo_coeff[:, orbital_ind]

In [ ]:
orbital_ind=3

print(np.isclose(F_MO[orbital_ind,orbital_ind], full_system_scf.mo_energy[orbital_ind]))

np.allclose(F_MO @ full_system_scf.mo_coeff[:, orbital_ind],
            full_system_scf.mo_coeff[:, orbital_ind]/full_system_scf.mo_energy[orbital_ind])


In [ ]:
orbital_ind=3
projector @ full_system_scf.mo_coeff[:, orbital_ind]

In [ ]:
C_all_localized[:, orbital_ind]

In [ ]:
Fock = full_system_scf.get_hcore() + full_system_scf.get_veff(dm = dm_active+ dm_enviro)
projector = np.zeros_like(Fock)
for orbital_ind in range(C_all_localized.shape[1]):
    if orbital_ind in enviro_MO_inds:
        outer = np.outer(psi_MO, psi_MO)
        projector-= Fock@outer + outer@Fock
    else:
       # inactive 
        continue

In [ ]:
orbital_ind=0
C_all_localized[:, orbital_ind].conj().T @ (Fock+projector)@ C_all_localized[:, orbital_ind]

In [ ]:
active_MO_inds

In [ ]:
Fock = full_system_scf.get_hcore() + full_system_scf.get_veff(dm = dm_active+ dm_enviro)

FP = (Fock + projector)

orbital_ind=2
psi_MO = EMBEDDED_full_system_scf.mo_coeff[:, orbital_ind]
psi_MO.conj().T @FP@psi_MO


In [ ]:
# psi_MO = EMBEDDED_occ_orbs[:, 1]

# P_B = 1e6* S_mat@ dm_enviro @ S_mat
# P_B = -0.5*(Fock@dm_enviro@S_mat + S_mat@dm_enviro@Fock)

for orbital_ind in range(EMBEDDED_full_system_scf.mo_coeff.shape[1]):
    psi_MO = EMBEDDED_full_system_scf.mo_coeff[:, orbital_ind]
    print(f'orb ind {orbital_ind}')
#     print(f'<MO| Vemb |MO > : {2*(psi_MO @ V_embed @ psi_MO)} \n')
    print(f'<MO| Pb |MO > : {2*(psi_MO @ P @ psi_MO)} \n')

In [ ]:
# Fock = full_system_scf.get_hcore() + full_system_scf.get_veff(dm = dm_active+ dm_enviro)
# S_mat = full_system_scf.get_ovlp()

# Fock_A = full_system_scf.get_hcore() + full_system_scf.get_veff(dm = dm_active)

In [ ]:
# F_A = Fock[:ao_active_inds[-1], :ao_active_inds[-1]]

# gamma_tot = 2 * C_all_localized @ C_all_localized.conj().T

# # gamma_act = gamma_tot[:ao_active_inds[-1]+1, :ao_active_inds[-1]+1]
# # gamma_env = gamma_tot[ao_ENViro_inds[0]:, ao_ENViro_inds[0]:]
# gamma_act = gamma_tot.copy()
# gamma_act[ao_ENViro_inds[0]:, ao_ENViro_inds[0]:] = np.zeros((len(ao_ENViro_inds), len(ao_ENViro_inds)))

# gamma_env = gamma_tot.copy()
# gamma_env[:ao_active_inds[-1]+1, :ao_active_inds[-1]+1] = np.zeros((len(ao_active_inds), len(ao_active_inds)))

# dm_active = gamma_act
# gamma_env = gamma_env
# V_embed  = vqe_in_dft.Geat_embedded_potential_operator(projector_method, 
#                                 full_system_scf, 
#                                 dm_active, 
#                                 dm_enviro, 
#                                 check_Hcore_is_correct=True, 
#                                 mu_shift_val=1e6,
#                                 check_Vemb=True)

In [ ]:
Fock = full_system_scf.get_hcore() + full_system_scf.get_veff(dm = dm_active+ dm_enviro)
S_mat = full_system_scf.get_ovlp()

Fock_A = full_system_scf.get_hcore() + full_system_scf.get_veff(dm = dm_active)

In [ ]:
# # psi_MO = EMBEDDED_occ_orbs[:, 1]

# P_B = 1e6* S_mat@ dm_enviro @ S_mat
# # P_B = -0.5*(Fock@dm_enviro@S_mat + S_mat@dm_enviro@Fock)

# for orbital_ind in range(EMBEDDED_full_system_scf.mo_coeff.shape[1]):
#     psi_MO = EMBEDDED_full_system_scf.mo_coeff[:, orbital_ind]
#     print(f'orb ind {orbital_ind}')
# #     print(f'<MO| Vemb |MO > : {2*(psi_MO @ V_embed @ psi_MO)} \n')
#     print(f'<MO| Pb |MO > : {2*(psi_MO @ P_B @ psi_MO)} \n')

In [ ]:
# np.around(Fock_A@P_B - P_B@Fock_A, 5)

# 5. Run RKS DFT of full system with $V_{emb}$ to get $\gamma_{emb}^{\text{active}}$ 

In [ ]:
full_system_mol_EMBEDDED = gto.Mole(atom= geometry,
                      basis=basis,
                       charge=charge,
                       spin=spin,
                      )
full_system_mol_EMBEDDED.build()

# RE-DEFINE number of electrons in system
full_system_mol_EMBEDDED.nelectron = 2*len(active_MO_inds) # <------ IMPORTANT!

EMBEDDED_full_system_scf = scf.RKS(full_system_mol_EMBEDDED)
EMBEDDED_full_system_scf.verbose=1
EMBEDDED_full_system_scf.max_memory= memory
EMBEDDED_full_system_scf.conv_tol = 1e-6
EMBEDDED_full_system_scf.xc = low_level_xc_functional

h_core = EMBEDDED_full_system_scf.get_hcore()

# overwrite h_core to include embedding term!!!!
EMBEDDED_full_system_scf.get_hcore = lambda *args: V_embed + h_core

E_emb = EMBEDDED_full_system_scf.kernel()

print(f'embedded Energy: {E_emb}')

In [ ]:
EMBEDDED_full_system_scf.conv_check

In [ ]:
EMBEDDED_occ_orbs = EMBEDDED_full_system_scf.mo_coeff[:,EMBEDDED_full_system_scf.mo_occ>0]

# optimized embedded denisty matrix
density_emb = 2 * EMBEDDED_occ_orbs @ EMBEDDED_occ_orbs.conj().T

## check number of electrons makes sense:
electron_check = np.isclose(np.trace(density_emb@full_system_scf.get_ovlp()), 2*len(active_MO_inds))

print(f'number of e- in gamma_embedded is correct: {electron_check}')

In [ ]:
# calculate embedding correction term

dm_correction = np.einsum('ij, ij', V_embed, density_emb-dm_active)
WF_correction = np.einsum('ij, ij', V_embed, dm_active)

print(f'RKS correction: {dm_correction}')
print(f'WF correction: {WF_correction}')

In [ ]:
## PsiEmbed Way

# J_emb, K_emb =EMBEDDED_full_system_scf.get_jk(dm=density_emb) 

# # note this uses the STANDARD H_core
# matrix_dot = lambda A, B: np.einsum('ij,ij', A, B)
# e_act_emb = matrix_dot(density_emb, h_core + 0.5 * J_emb - 0.25 * K_emb)
# e_act_emb

## MY way
e_act_emb = full_system_scf.energy_elec(dm=density_emb,
                                        vhf= full_system_scf.get_veff(dm=density_emb),
                                       h1e = h_core)[0]
e_act_emb

In [ ]:
e_mf_emb = e_act_emb + E_env + two_e_cross + full_system_scf.energy_nuc() + dm_correction
e_mf_emb # <-- energy from embedded DFT calc

In [ ]:
full_system_scf.e_tot

In [ ]:
print(f'global DFT calculation == seperated calculation: {np.isclose(e_mf_emb, full_system_scf.e_tot)}')
# expected as same functional used!

# 6. High level DFT calc!

In [ ]:
full_system_mol_HIGH_LEVEL_DFT = gto.Mole(atom= geometry,
                      basis=basis,
                       charge=charge,
                       spin=spin,
                      )

full_system_mol_HIGH_LEVEL_DFT.nelectron = 2*len(active_MO_inds) # <------ IMPORTANT!
full_system_mol_HIGH_LEVEL_DFT.build()

full_system_scf_HIGH_LEVEL = scf.RKS(full_system_mol_HIGH_LEVEL_DFT)
full_system_scf_HIGH_LEVEL.verbose=1
full_system_scf_HIGH_LEVEL.max_memory= memory
full_system_scf_HIGH_LEVEL.conv_tol = 1e-6
full_system_scf_HIGH_LEVEL.xc = high_level_xc_functional # <-- BETTER functional!


# full_system_scf_HIGH_LEVEL.kernel() < --- do NOT RUN THIS

In [ ]:
# run energy calc using high level functional (note density matrix is embedded A)

e_act_emb_HIGH_LVL = full_system_scf_HIGH_LEVEL.energy_elec(dm=density_emb)
e_act_emb_HIGH_LVL 

In [ ]:
E_high_lvl_DFT = e_act_emb_HIGH_LVL[0] + E_env + two_e_cross + full_system_scf.energy_nuc() + dm_correction
E_high_lvl_DFT # <-- energy from embedded DFT calc

In [ ]:
print('High level DFT in DFT error:', np.abs(E_high_lvl_DFT-my_fci.e_tot))
print('LOW level DFT in DFT error:', np.abs(e_mf_emb-my_fci.e_tot))

# 7. High level WF calc (classical run) !

In [ ]:
full_system_mol_EMBEDDED_HF = gto.Mole(atom= geometry,
                      basis=basis,
                       charge=charge,
                       spin=spin,
                      )
full_system_mol_EMBEDDED_HF.build()

# RE-DEFINE number of electrons in system
full_system_mol_EMBEDDED_HF.nelectron = 2*len(active_MO_inds) # <------ IMPORTANT!

EMBEDDED_full_system_scf_HF = scf.RHF(full_system_mol_EMBEDDED) # <---- Hartree Fock Calc!
EMBEDDED_full_system_scf_HF.verbose=1
EMBEDDED_full_system_scf_HF.max_memory= memory
EMBEDDED_full_system_scf_HF.conv_tol = 1e-6

h_core = EMBEDDED_full_system_scf_HF.get_hcore()

# overwrite h_core to include embedding term!!!!
EMBEDDED_full_system_scf_HF.get_hcore = lambda *args: V_embed + h_core

# EMBEDDED_full_system_scf_HF.kernel() # <------ do NOT RUN!

In [ ]:
# overwrite orbs with RKS embedded orbs!
EMBEDDED_full_system_scf_HF.mo_coeff = EMBEDDED_full_system_scf.mo_coeff 
EMBEDDED_full_system_scf_HF.mo_occ = EMBEDDED_full_system_scf.mo_occ 
EMBEDDED_full_system_scf_HF.mo_energy = EMBEDDED_full_system_scf.mo_energy

print(EMBEDDED_full_system_scf_HF.energy_elec(dm=density_emb)[0])
print(EMBEDDED_full_system_scf_HF.energy_tot(dm=density_emb))
print()
print(EMBEDDED_full_system_scf_HF.energy_elec()[0])
print(EMBEDDED_full_system_scf_HF.energy_tot())

In [ ]:
# EMBEDDED_full_system_scf_HF.kernel()
# FCI_embedded = fci.FCI(EMBEDDED_full_system_scf_HF)
# E_FCI,FCI_vec = FCI_embedded.kernel()
# # from pyscf.ci.cisd import  to_fcivec
# # cisdWFN   = to_fcivec(C_FCI, len(active_MO_inds), 2*len(active_MO_inds))

# RDM1 = FCI_embedded.make_rdm1(FCI_vec,
#                        norb=EMBEDDED_full_system_scf_HF.mo_coeff.shape[1],
#                        nelec=2*len(active_MO_inds))

# V,S, U = np.linalg.svd(RDM1)
# print(np.isclose(2*len(active_MO_inds), np.sum(S)))
# print(S)

# print('')
# print('ignore: ', S[])

In [ ]:
## CCSD calculation

embedded_cc_obj = cc.CCSD(EMBEDDED_full_system_scf_HF)

embedded_cc_obj.frozen = [i for i in range(EMBEDDED_full_system_scf_HF.mol.nao - len(enviro_MO_inds),
                                           EMBEDDED_full_system_scf_HF.mol.nao)
                         ]
# embedded_cc_obj.frozen  = enviro_MO_inds.tolist()

e_cc, t1, t2 = embedded_cc_obj.kernel()

CC_flag_check = np.isclose(EMBEDDED_full_system_scf_HF.energy_tot(dm=density_emb),
                          embedded_cc_obj.e_hf)


print(f'E_corr: {e_cc}')
print(f'\nCC hartree fock energy matches HF embedded calc: {CC_flag_check}!')
embedded_cc_obj.e_hf

In [ ]:
print('orbital energies:')
print(EMBEDDED_full_system_scf_HF.mo_energy)

In [ ]:
print(embedded_cc_obj.frozen)

In [ ]:
WF_correction  = np.einsum('ij, ij', V_embed, dm_active) # note different definition

E_WF = embedded_cc_obj.e_hf +e_cc  + E_env + two_e_cross - WF_correction
E_WF 

In [ ]:
print('High level DFT in DFT error:', np.abs(E_high_lvl_DFT-my_fci.e_tot))
print('LOW level DFT in DFT error:', np.abs(e_mf_emb-my_fci.e_tot))

print('WF in DFT error:', np.abs(E_WF-my_fci.e_tot))

In [ ]:
drive_obj = vqe_in_dft.embeddeding_SCF_driver(geometry,
                 N_active_atoms,
                 projector_method,
                 cheap_global_SCF_method='RKS', 
                 cheap_global_DFT_xc= 'lda, vwn',
                 expensive_global_DFT_xc = 'b3lyp',
                 cheap_WF_method = 'RHF',
                 expensive_WF_method = 'CCSD',
                 E_convergence_tol = 1e-6,
                 basis = basis,
                 unit= 'angstrom',
                 pyscf_print_level=1,
                 memory=8000,
                 charge=0,
                 spin=0,
                 run_fci=False,
                 run_cisd=False,
                 mu_value=1e6,
                 physists_notation= False)#phys_notation)

E_DFT_low, E_DFT_high, E_Classical, E_QC, H_ferm = drive_obj.run_experiment(localization_method, 
                                                                      orbtial_loc_threshold=THRESHOLD,
                                                                   jupyter_notebook=True)

In [ ]:
# np.binary_repr(52).count('1')  

In [ ]:
print('LOW level DFT in DFT error:', np.abs(E_DFT_low-my_fci.e_tot))
print('High level DFT in DFT error:', np.abs(E_DFT_high-my_fci.e_tot))
print('WF in DFT error:', np.abs(E_Classical-my_fci.e_tot))
print('VQE in DFT error:', np.abs(E_QC-my_fci.e_tot))

In [ ]:
print(e_mf_emb)
print(E_high_lvl_DFT)
print(E_WF)

# QC part

$$H_{fermionic} = h_{nuc} + \sum_{p, q} h_{pq} a_{p}^{\dagger} a_{q} + \frac{1}{2} \sum_{p, q ,r ,s} h_{pqrs} a_{p}^{\dagger} a_{q}^{\dagger} a_{r} a_{s}$$

where:

- $$h_{pq} = \int d{\vec{x}} \phi_{p}^{*}({\vec{x}}) \bigg( - \frac{\nabla^{2}_{{\vec{r}}}}{2} - \sum_{I} \frac{Z_{I}}{|{\vec{r}} - {\vec{R}_{I}} |} \bigg) \phi_{q}({\vec{x}})$$


- $$h_{pqrs} = \int d{\vec{x}}_{1} d{\vec{x}}_{2} \frac{\phi_{p}^{*}({\vec{x}}_{1}) \phi_{q}^{*}({\vec{x}}_{2}) \phi_{s}({\vec{x}}_{1}) \phi_{r}({\vec{x}}_{2})}{|{\vec{r}_{1}} - {\vec{r}}_{2}|}$$ 


- $$h_{nuc} = \frac{1}{2} \sum_{I \neq J} \frac{Z_{I} Z_{J}}{| {\vec{R}}_{I} - {\vec{R}}_{J}|}$$

note project means we don't want last set of MO orbs filled!

In [ ]:
# EMBEDDED_full_system_scf_HF.mo_coeff[:]

In [ ]:
N_enviroment_MOs = len(enviro_MO_inds) 

one_body_integrals, two_body_integrals = vqe_in_dft.Get_embedded_one_and_two_body_integrals_MO_basis(EMBEDDED_full_system_scf_HF,
                                                                                            N_enviroment_MOs,
                                                                                            physists_notation=phys_notation)

In [ ]:
print(one_body_integrals.shape)
print(two_body_integrals.shape)

print(active_MO_inds)
print(enviro_MO_inds)

In [ ]:
one_body_terms, two_body_terms = vqe_in_dft.Get_SpinOrbs_from_Spatial(one_body_integrals,
                                                                   two_body_integrals,
                                                                   physists_notation=phys_notation,
                                                                   EQ_Tolerance=1e-8)

In [ ]:
Nuclear_energy =  full_system_scf.energy_nuc()

H_fermionic = vqe_in_dft.Get_fermionic_H(one_body_terms, 
                                     two_body_terms, 
                                     Nuclear_energy,
                                     core_constant=0, 
                                     physists_notation=phys_notation,
                                        jupyter_notebook=True)

In [ ]:
len(list(H_fermionic))

In [ ]:
from openfermion.linalg import get_sparse_operator
H_sparse = get_sparse_operator(H_fermionic)

In [ ]:
eigvals_EMBED, eigvecs_EMBED = sp.sparse.linalg.eigsh(H_sparse, which='SA', k=1)
eigvals_EMBED

In [ ]:
WF_correction  = np.einsum('ij, ij', V_embed, dm_active) # note different definition

E_VQE = eigvals_EMBED[0]  + E_env + two_e_cross - WF_correction
E_VQE 

In [ ]:
N_electrons_expected = 2*len(active_MO_inds)
N_electrons_Q_state = np.binary_repr(np.where(np.abs(eigvecs_EMBED)>1e-2)[0][0]).count('1') 

print(f'expect {N_electrons_expected} electrons')
print(f'quantum state has {N_electrons_Q_state} electrons \n')

print(f'number of electrons correct: {N_electrons_expected == N_electrons_Q_state}')

In [ ]:
print('High level DFT in DFT error:', np.abs(E_high_lvl_DFT-my_fci.e_tot))
print('LOW level DFT in DFT error:', np.abs(e_mf_emb-my_fci.e_tot))


print('WF in DFT error:', np.abs(E_WF-my_fci.e_tot))
print('VQE error:', np.abs(E_VQE-my_fci.e_tot))

In [ ]:
where=np.where(np.around(eigvecs_EMBED, 4)>0)[0]
print(where)

np.binary_repr(where[0]).count('1') 